# Libs

In [836]:
import pandas as pd
import numpy as np
import sqlite3

# show all available columns
pd.set_option('display.max_columns', 500)
# show all available rows
pd.set_option('display.max_rows', 500)

# Data Prep
from feature_engine.categorical_encoders import CountFrequencyCategoricalEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from category_encoders import TargetEncoder

# boruta
from boruta import BorutaPy
from pycaret.datasets import get_data
from sklearn.feature_selection import RFECV, f_classif, chi2

# import classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import auc, precision_recall_curve, log_loss, recall_score, precision_score, make_scorer

# Validation
from sklearn.model_selection import learning_curve, KFold, LeaveOneOut, cross_validate, validation_curve

# models
import statsmodels.formula.api as smf
import statsmodels.api as sm

# regression metrics
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score
from sklearn.metrics import classification_report, f1_score, fbeta_score
from sklearn.metrics import accuracy_score, recall_score, precision_score, classification_report

# confusion matrix  
from sklearn.model_selection import cross_val_predict, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, log_loss, make_scorer

# warnig treatments
import warnings
warnings.filterwarnings('ignore')

# ensemble models 
from sklearn.ensemble import AdaBoostClassifier, VotingClassifier, StackingClassifier, ExtraTreesClassifier
from xgboost import XGBRegressor, XGBClassifier

# neural network model
from sklearn.neural_network import MLPClassifier

# statistics
from scipy.stats import loguniform, uniform

# DB Connection - Premier Leagues DB

In [29]:
# conexão com o BD Premier League
db_PremierLeague = sqlite3.connect('PremierLeague.db')

# Main Query 

In [707]:
query01 = """
SELECT 'Results_2001' as Season, ('20' || date) as Game_Date, HomeTeam, AwayTeam, FTR, FTHG, FTAG, HTR, HTHG, HTAG, HST, AST, HF, AF, HY, AY, HR, AR, 
(GBH + IWH + LBH + SBH + WHH) / ( (GBH + IWH + LBH + SBH + WHH) + (GBD + IWD + LBD + SBD + WHD) + (GBA + IWA + LBA + SBA + WHA) ) as Home_Mean_Bet,
(GBD + IWD + LBD + SBD + WHD) / ( (GBH + IWH + LBH + SBH + WHH) + (GBD + IWD + LBD + SBD + WHD) + (GBA + IWA + LBA + SBA + WHA) ) as Draw_Mean_Bet,
(GBA + IWA + LBA + SBA + WHA) / ( (GBH + IWH + LBH + SBH + WHH) + (GBD + IWD + LBD + SBD + WHD) + (GBA + IWA + LBA + SBA + WHA) ) as Away_Mean_Bet
From Results_2001
UNION
SELECT 'Results_2002' as Season, ('20' || date) as Game_Date, HomeTeam, AwayTeam, FTR, FTHG, FTAG, HTR, HTHG, HTAG, HST, AST, HF, AF, HY, AY, HR, AR, 
(GBH + IWH + LBH + SBH + WHH + SYH) / ( (GBH + IWH + LBH + SBH + WHH + SYH) + (GBD + IWD + LBD + SBD + WHD + SYD) + (GBA + IWA + LBA + SBA + WHA + SYA) ) as Home_Mean_Bet,
(GBD + IWD + LBD + SBD + WHD + SYD) / ( (GBH + IWH + LBH + SBH + WHH + SYH) + (GBD + IWD + LBD + SBD + WHD + SYD) + (GBA + IWA + LBA + SBA + WHA + SYA) ) as Draw_Mean_Bet,
(GBA + IWA + LBA + SBA + WHA + SYA) / ( (GBH + IWH + LBH + SBH + WHH + SYH) + (GBD + IWD + LBD + SBD + WHD + SYD) + (GBA + IWA + LBA + SBA + WHA + SYA) ) as Away_Mean_Bet
From Results_2002
UNION
SELECT 'Results_2003' as Season, ('20' || date) as Game_Date, HomeTeam, AwayTeam, FTR, FTHG, FTAG, HTR, HTHG, HTAG, HST, AST, HF, AF, HY, AY, HR, AR, 
(B365H + GBH + IWH + LBH + SOH + SBH + WHH) / ((B365H + GBH + IWH + LBH + SOH + SBH + WHH) + (B365D + GBD + IWD + LBD + SOD + SBD + WHD) + (B365A + GBA + IWA + LBA + SOA + SBA + WHA)) as Home_Mean_Bet,
(B365D + GBD + IWD + LBD + SOD + SBD + WHD) / ((B365H + GBH + IWH + LBH + SOH + SBH + WHH) + (B365D + GBD + IWD + LBD + SOD + SBD + WHD) + (B365A + GBA + IWA + LBA + SOA + SBA + WHA)) as Draw_Mean_Bet,
(B365A + GBA + IWA + LBA + SOA + SBA + WHA) / ((B365H + GBH + IWH + LBH + SOH + SBH + WHH) + (B365D + GBD + IWD + LBD + SOD + SBD + WHD) + (B365A + GBA + IWA + LBA + SOA + SBA + WHA)) as Away_Mean_Bet
From Results_2003
UNION
SELECT 'Results_2004' as Season, ('20' || date) as Game_Date, HomeTeam, AwayTeam, FTR, FTHG, FTAG, HTR, HTHG, HTAG, HST, AST, HF, AF, HY, AY, HR, AR, 
(B365H + GBH + IWH + LBH + SOH + SBH + WHH) / ((B365H + GBH + IWH + LBH + SOH + SBH + WHH) + (B365D + GBD + IWD + LBD + SOD + SBD + WHD) + (B365A + GBA + IWA + LBA + SOA + SBA + WHA)) as Home_Mean_Bet,
(B365D + GBD + IWD + LBD + SOD + SBD + WHD) / ((B365H + GBH + IWH + LBH + SOH + SBH + WHH) + (B365D + GBD + IWD + LBD + SOD + SBD + WHD) + (B365A + GBA + IWA + LBA + SOA + SBA + WHA)) as Draw_Mean_Bet,
(B365A + GBA + IWA + LBA + SOA + SBA + WHA) / ((B365H + GBH + IWH + LBH + SOH + SBH + WHH) + (B365D + GBD + IWD + LBD + SOD + SBD + WHD) + (B365A + GBA + IWA + LBA + SOA + SBA + WHA)) as Away_Mean_Bet
From Results_2004
UNION
SELECT 'Results_2005' as Season, ('20' || date) as Game_Date, HomeTeam, AwayTeam, FTR, FTHG, FTAG, HTR, HTHG, HTAG, HST, AST, HF, AF, HY, AY, HR, AR, 
(B365H + GBH + IWH + LBH + SBH + WHH) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Home_Mean_Bet,
(B365D + GBD + IWD + LBD + SBD + WHD) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Draw_Mean_Bet,
(B365A + GBA + IWA + LBA + SBA + WHA) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Away_Mean_Bet
From Results_2005
UNION
SELECT 'Results_2006' as Season, ('20' || date) as Game_Date, HomeTeam, AwayTeam, FTR, FTHG, FTAG, HTR, HTHG, HTAG, HST, AST, HF, AF, HY, AY, HR, AR, 
(B365H + GBH + IWH + LBH + SBH + WHH) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Home_Mean_Bet,
(B365D + GBD + IWD + LBD + SBD + WHD) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Draw_Mean_Bet,
(B365A + GBA + IWA + LBA + SBA + WHA) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Away_Mean_Bet
From Results_2006
UNION
SELECT 'Results_2007' as Season, ('20' || date) as Game_Date, HomeTeam, AwayTeam, FTR, FTHG, FTAG, HTR, HTHG, HTAG, HST, AST, HF, AF, HY, AY, HR, AR, 
(B365H + GBH + IWH + LBH + SBH + WHH) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Home_Mean_Bet,
(B365D + GBD + IWD + LBD + SBD + WHD) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Draw_Mean_Bet,
(B365A + GBA + IWA + LBA + SBA + WHA) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Away_Mean_Bet
From Results_2007
UNION
SELECT 'Results_2008' as Season, ('20' || date) as Game_Date, HomeTeam, AwayTeam, FTR, FTHG, FTAG, HTR, HTHG, HTAG, HST, AST, HF, AF, HY, AY, HR, AR, 
(B365H + GBH + IWH + LBH + SBH + WHH) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Home_Mean_Bet,
(B365D + GBD + IWD + LBD + SBD + WHD) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Draw_Mean_Bet,
(B365A + GBA + IWA + LBA + SBA + WHA) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Away_Mean_Bet
From Results_2008
UNION
SELECT 'Results_2009' as Season, ('20' || date) as Game_Date, HomeTeam, AwayTeam, FTR, FTHG, FTAG, HTR, HTHG, HTAG, HST, AST, HF, AF, HY, AY, HR, AR, 
(B365H + GBH + IWH + LBH + SBH + WHH) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Home_Mean_Bet,
(B365D + GBD + IWD + LBD + SBD + WHD) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Draw_Mean_Bet,
(B365A + GBA + IWA + LBA + SBA + WHA) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Away_Mean_Bet
From Results_2009
UNION
SELECT 'Results_2010' as Season, ('20' || date) as Game_Date, HomeTeam, AwayTeam, FTR, FTHG, FTAG, HTR, HTHG, HTAG, HST, AST, HF, AF, HY, AY, HR, AR, 
(B365H + GBH + IWH + LBH + SBH + WHH) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Home_Mean_Bet,
(B365D + GBD + IWD + LBD + SBD + WHD) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Draw_Mean_Bet,
(B365A + GBA + IWA + LBA + SBA + WHA) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Away_Mean_Bet
From Results_2010
UNION
SELECT 'Results_2011' as Season, ('20' || date) as Game_Date, HomeTeam, AwayTeam, FTR, FTHG, FTAG, HTR, HTHG, HTAG, HST, AST, HF, AF, HY, AY, HR, AR, 
(B365H + GBH + IWH + LBH + SBH + WHH) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Home_Mean_Bet,
(B365D + GBD + IWD + LBD + SBD + WHD) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Draw_Mean_Bet,
(B365A + GBA + IWA + LBA + SBA + WHA) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Away_Mean_Bet
From Results_2011
UNION
SELECT 'Results_2012' as Season, ('20' || date) as Game_Date, HomeTeam, AwayTeam, FTR, FTHG, FTAG, HTR, HTHG, HTAG, HST, AST, HF, AF, HY, AY, HR, AR, 
(B365H + GBH + IWH + LBH + SBH + WHH) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Home_Mean_Bet,
(B365D + GBD + IWD + LBD + SBD + WHD) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Draw_Mean_Bet,
(B365A + GBA + IWA + LBA + SBA + WHA) / ((B365H + GBH + IWH + LBH + SBH + WHH) + (B365D + GBD + IWD + LBD + SBD + WHD) + (B365A + GBA + IWA + LBA + SBA + WHA)) as Away_Mean_Bet
From Results_2012
UNION
SELECT 'Results_2013' as Season, ('20' || date) as Game_Date, HomeTeam, AwayTeam, FTR, FTHG, FTAG, HTR, HTHG, HTAG, HST, AST, HF, AF, HY, AY, HR, AR, 
(B365H + GBH + IWH + LBH + PSH + WHH + VCH) / ((B365H + GBH + IWH + LBH + PSH + WHH + VCH) + (B365D + GBD + IWD + LBD + PSD + WHD + VCD) + (B365A + GBA + IWA + LBA + PSA + WHA + VCA)) as Home_Mean_Bet,
(B365D + GBD + IWD + LBD + PSD + WHD + VCD) / ((B365H + GBH + IWH + LBH + PSH + WHH + VCH) + (B365D + GBD + IWD + LBD + PSD + WHD + VCD) + (B365A + GBA + IWA + LBA + PSA + WHA + VCA)) as Draw_Mean_Bet,
(B365A + GBA + IWA + LBA + PSA + WHA + VCA) / ((B365H + GBH + IWH + LBH + PSH + WHH + VCH) + (B365D + GBD + IWD + LBD + PSD + WHD + VCD) + (B365A + GBA + IWA + LBA + PSA + WHA + VCA)) as Away_Mean_Bet
From Results_2013
UNION
SELECT 'Results_2014' as Season, ('20' || date) as Game_Date, HomeTeam, AwayTeam, FTR, FTHG, FTAG, HTR, HTHG, HTAG, HST, AST, HF, AF, HY, AY, HR, AR, 
(B365H + PSCH + IWH + LBH + PSH + WHH + VCH) / ((B365H + PSCH + IWH + LBH + PSH + WHH + VCH) + (B365D + PSCD + IWD + LBD + PSD + WHD + VCD) + (B365A + PSCA + IWA + LBA + PSA + WHA + VCA)) as Home_Mean_Bet,
(B365D + PSCD + IWD + LBD + PSD + WHD + VCD) / ((B365H + PSCH + IWH + LBH + PSH + WHH + VCH) + (B365D + PSCD + IWD + LBD + PSD + WHD + VCD) + (B365A + PSCA + IWA + LBA + PSA + WHA + VCA)) as Draw_Mean_Bet,
(B365A + PSCA + IWA + LBA + PSA + WHA + VCA) / ((B365H + PSCH + IWH + LBH + PSH + WHH + VCH) + (B365D + PSCD + IWD + LBD + PSD + WHD + VCD) + (B365A + PSCA + IWA + LBA + PSA + WHA + VCA)) as Away_Mean_Bet
From Results_2014
UNION
SELECT 'Results_2015' as Season, ('20' || date) as Game_Date, HomeTeam, AwayTeam, FTR, FTHG, FTAG, HTR, HTHG, HTAG, HST, AST, HF, AF, HY, AY, HR, AR, 
(B365H + PSCH + IWH + LBH + PSH + WHH + VCH) / ((B365H + PSCH + IWH + LBH + PSH + WHH + VCH) + (B365D + PSCD + IWD + LBD + PSD + WHD + VCD) + (B365A + PSCA + IWA + LBA + PSA + WHA + VCA)) as Home_Mean_Bet,
(B365D + PSCD + IWD + LBD + PSD + WHD + VCD) / ((B365H + PSCH + IWH + LBH + PSH + WHH + VCH) + (B365D + PSCD + IWD + LBD + PSD + WHD + VCD) + (B365A + PSCA + IWA + LBA + PSA + WHA + VCA)) as Draw_Mean_Bet,
(B365A + PSCA + IWA + LBA + PSA + WHA + VCA) / ((B365H + PSCH + IWH + LBH + PSH + WHH + VCH) + (B365D + PSCD + IWD + LBD + PSD + WHD + VCD) + (B365A + PSCA + IWA + LBA + PSA + WHA + VCA)) as Away_Mean_Bet
From Results_2015
UNION
SELECT 'Results_2016' as Season, ('20' || date) as Game_Date, HomeTeam, AwayTeam, FTR, FTHG, FTAG, HTR, HTHG, HTAG, HST, AST, HF, AF, HY, AY, HR, AR, 
(B365H + PSCH + IWH + LBH + PSH + WHH + VCH) / ((B365H + PSCH + IWH + LBH + PSH + WHH + VCH) + (B365D + PSCD + IWD + LBD + PSD + WHD + VCD) + (B365A + PSCA + IWA + LBA + PSA + WHA + VCA)) as Home_Mean_Bet,
(B365D + PSCD + IWD + LBD + PSD + WHD + VCD) / ((B365H + PSCH + IWH + LBH + PSH + WHH + VCH) + (B365D + PSCD + IWD + LBD + PSD + WHD + VCD) + (B365A + PSCA + IWA + LBA + PSA + WHA + VCA)) as Draw_Mean_Bet,
(B365A + PSCA + IWA + LBA + PSA + WHA + VCA) / ((B365H + PSCH + IWH + LBH + PSH + WHH + VCH) + (B365D + PSCD + IWD + LBD + PSD + WHD + VCD) + (B365A + PSCA + IWA + LBA + PSA + WHA + VCA)) as Away_Mean_Bet
From Results_2016
UNION
SELECT 'Results_2017' as Season, ('20' || date) as Game_Date, HomeTeam, AwayTeam, FTR, FTHG, FTAG, HTR, HTHG, HTAG, HST, AST, HF, AF, HY, AY, HR, AR, 
(B365H + PSCH + IWH + LBH + PSH + WHH + VCH) / ((B365H + PSCH + IWH + LBH + PSH + WHH + VCH) + (B365D + PSCD + IWD + LBD + PSD + WHD + VCD) + (B365A + PSCA + IWA + LBA + PSA + WHA + VCA)) as Home_Mean_Bet,
(B365D + PSCD + IWD + LBD + PSD + WHD + VCD) / ((B365H + PSCH + IWH + LBH + PSH + WHH + VCH) + (B365D + PSCD + IWD + LBD + PSD + WHD + VCD) + (B365A + PSCA + IWA + LBA + PSA + WHA + VCA)) as Draw_Mean_Bet,
(B365A + PSCA + IWA + LBA + PSA + WHA + VCA) / ((B365H + PSCH + IWH + LBH + PSH + WHH + VCH) + (B365D + PSCD + IWD + LBD + PSD + WHD + VCD) + (B365A + PSCA + IWA + LBA + PSA + WHA + VCA)) as Away_Mean_Bet
From Results_2017
UNION
SELECT 'Results_2018' as Season, ('20' || date) as Game_Date, HomeTeam, AwayTeam, FTR, FTHG, FTAG, HTR, HTHG, HTAG, HST, AST, HF, AF, HY, AY, HR, AR, 
(B365H + PSCH + IWH + LBH + PSH + WHH + VCH) / ((B365H + PSCH + IWH + LBH + PSH + WHH + VCH) + (B365D + PSCD + IWD + LBD + PSD + WHD + VCD) + (B365A + PSCA + IWA + LBA + PSA + WHA + VCA)) as Home_Mean_Bet,
(B365D + PSCD + IWD + LBD + PSD + WHD + VCD) / ((B365H + PSCH + IWH + LBH + PSH + WHH + VCH) + (B365D + PSCD + IWD + LBD + PSD + WHD + VCD) + (B365A + PSCA + IWA + LBA + PSA + WHA + VCA)) as Draw_Mean_Bet,
(B365A + PSCA + IWA + LBA + PSA + WHA + VCA) / ((B365H + PSCH + IWH + LBH + PSH + WHH + VCH) + (B365D + PSCD + IWD + LBD + PSD + WHD + VCD) + (B365A + PSCA + IWA + LBA + PSA + WHA + VCA)) as Away_Mean_Bet
From Results_2018
UNION
SELECT 'Results_2019' as Season, ('20' || date) as Game_Date, HomeTeam, AwayTeam, FTR, FTHG, FTAG, HTR, HTHG, HTAG, HST, AST, HF, AF, HY, AY, HR, AR, 
(B365H + PSCH + IWH + PSH + WHH + VCH) / ((B365H + PSCH + IWH + PSH + WHH + VCH) + (B365D + PSCD + IWD + PSD + WHD + VCD) + (B365A + PSCA + IWA + PSA + WHA + VCA)) as Home_Mean_Bet,
(B365D + PSCD + IWD + PSD + WHD + VCD) / ((B365H + PSCH + IWH + PSH + WHH + VCH) + (B365D + PSCD + IWD + PSD + WHD + VCD) + (B365A + PSCA + IWA + PSA + WHA + VCA)) as Draw_Mean_Bet,
(B365A + PSCA + IWA + PSA + WHA + VCA) / ((B365H + PSCH + IWH + PSH + WHH + VCH) + (B365D + PSCD + IWD + PSD + WHD + VCD) + (B365A + PSCA + IWA + PSA + WHA + VCA)) as Away_Mean_Bet
From Results_2019
ORDER by Game_Date

          """
df_allresults_Season = pd.read_sql_query(query01, db_PremierLeague)
df_allresults_Season.head(3)

,Season,Game_Date,HomeTeam,AwayTeam,FTR,FTHG,FTAG,HTR,HTHG,HTAG,HST,AST,HF,AF,HY,AY,HR,AR,Home_Mean_Bet,Draw_Mean_Bet,Away_Mean_Bet
0,Results_2001,2000-08-19,Charlton,Man City,H,4,0,H,2,0,14,4,13,12,1,2,0,0,0.261422,0.381139,0.357440
1,Results_2001,2000-08-19,Chelsea,West Ham,H,4,2,H,1,0,10,5,19,14,1,2,0,0,0.142749,0.326934,0.530317
2,Results_2001,2000-08-19,Coventry,Middlesbrough,A,1,3,D,1,1,3,9,15,21,5,3,1,0,0.276407,0.382527,0.341066


## Feature engineering within original dataset

In [708]:
# if it was finished the game with the same result that ended the half time, just for winnings - 'H' Home
df_allresults_Season['Home_Win_Half_Full'] = np.where( (df_allresults_Season['FTR'] == df_allresults_Season['HTR']) \
   & (df_allresults_Season['FTR'] == 'H'), 1, 0)

In [709]:
# if it was finished the game with the same result that ended the half time, just for winnings - 'A' Away
df_allresults_Season['Away_Win_Half_Full'] = np.where( (df_allresults_Season['FTR'] == df_allresults_Season['HTR']) \
   & (df_allresults_Season['FTR'] == 'A'), 1, 0)

In [710]:
# if opponent team got more red cards, then the other team would win - Home
df_allresults_Season['Home_Win_Red_Against'] = np.where( ((df_allresults_Season['FTR'] == 'H')  \
    & (df_allresults_Season['AR'] > 0) \
    & (df_allresults_Season['HR'] < df_allresults_Season['AR']) ) , 1, 0)

In [711]:
# if opponent team got more red cards, then the other team would win - Away
df_allresults_Season['Away_Win_Red_Against'] = np.where( ((df_allresults_Season['FTR'] == 'A') \
    & (df_allresults_Season['HR'] > 0) 
    & (df_allresults_Season['HR'] > df_allresults_Season['AR'])), 1, 0)

In [712]:
df_allresults_Season[['HomeTeam', 'AwayTeam', 'FTR', 'HTR', 'HR', 'AR', 'Home_Win_Red_Against', 'Away_Win_Red_Against']].\
loc[ ((df_allresults_Season['FTR'] == 'A') \
    & (df_allresults_Season['HR'] > 0) 
    & (df_allresults_Season['HR'] > df_allresults_Season['AR'])) \
   | ((df_allresults_Season['FTR'] == 'H')  \
    & (df_allresults_Season['AR'] > 0) \
    & (df_allresults_Season['HR'] < df_allresults_Season['AR']) ) ]

,HomeTeam,AwayTeam,FTR,HTR,HR,AR,Home_Win_Red_Against,Away_Win_Red_Against
2,Coventry,Middlesbrough,A,D,1,0,0,1
7,Sunderland,Arsenal,H,D,0,1,1,0
10,Arsenal,Liverpool,H,H,1,2,1,0
14,Everton,Charlton,H,D,0,1,1,0
18,West Ham,Leicester,A,D,1,0,0,1
...,...,...,...,...,...,...,...,...
7166,Brighton,Bournemouth,A,A,1,0,0,1
7174,Watford,Arsenal,A,A,1,0,0,1
7198,Leicester,Arsenal,H,D,0,1,1,0
7201,Bournemouth,Tottenham,H,D,0,2,1,0


In [713]:
df_allresults_Season.sample(3)

,Season,Game_Date,HomeTeam,AwayTeam,FTR,FTHG,FTAG,HTR,HTHG,HTAG,HST,AST,HF,AF,HY,AY,HR,AR,Home_Mean_Bet,Draw_Mean_Bet,Away_Mean_Bet,Home_Win_Half_Full,Away_Win_Half_Full,Home_Win_Red_Against,Away_Win_Red_Against
2131,Results_2006,2006-02-01,Liverpool,Birmingham,D,1,1,D,0,0,10,2,13,12,0,0,0,1,0.083226,0.289361,0.627413,0,0,0,0
6272,Results_2017,2017-01-02,Middlesbrough,Leicester,D,0,0,D,0,0,1,4,10,9,3,0,0,0,0.311475,0.358197,0.330328,0,0,0,0
67,Results_2001,2000-09-23,Tottenham,Man City,D,0,0,D,0,0,5,2,10,14,0,2,0,0,0.157450,0.345868,0.496682,0,0,0,0


# Saving main dataset and including into Premier League DB

In [714]:
df_allresults_Season.to_csv('PremierLeague_Allresults_Season_2nd.csv', index=False)

# Query for mobile past 05 games

>> The target of this query is to leverage the last 05 results of the team (home and away respectively) in the league history

In [715]:
# query mean mobile from 2001 until 2019 for past 05 games

query02 = """

Select Season, Game_Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR as Result, HTHG, HTAG, HTR as HaltTime_Result,  
       -- Home Team mean mobile goals 
	   AVG(FTHG)
	   OVER( PARTITION BY HomeTeam
			 ORDER BY Game_Date 
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Home_Mean_Goals,
       -- Away Team mean mobile goals 
	   AVG(FTAG)
	   OVER( PARTITION BY AwayTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Away_Mean_Goals,
       -- Home Team mean mobile half time goals 
	   AVG(HTHG)
	   OVER( PARTITION BY HomeTeam
			 ORDER BY Game_Date 
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Home_Mean_HT_Goals,
       -- Away Team mean mobile half time goals goals 
	   AVG(HTAG)
	   OVER( PARTITION BY AwayTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Away_Mean_HT_Goals,
		   -- Home Team mean mobile shots on target 
	   AVG(HST)
	   OVER( PARTITION BY HomeTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Home_Mean_Shots_Target,
       -- Away Team mean mobile shots on target 
	   AVG(AST)
	   OVER( PARTITION BY AwayTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Away_Mean_Shots_Target,
	   -- Home Team mean mobile goals per shot 	   
	   AVG(HST/FTHG)
	   OVER( PARTITION BY HomeTeam
			 ORDER BY Game_Date DESC
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING  
		   ) AS Home_Mean_Shots_per_Goals,
       -- Away Team mean mobile goals per shot
	   AVG(AST/FTAG)
	   OVER( PARTITION BY AwayTeam
			 ORDER BY Game_Date DESC
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Away_Mean_Shots_per_Goals,
	   AVG(HF)
	   OVER( PARTITION BY HomeTeam
			 ORDER BY Game_Date 
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Home_Mean_Faults_Committed,
       -- Away Team mean mobile faults commited
	   AVG(AF)
	   OVER( PARTITION BY AwayTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Away_Mean_Faults_Committed,
	   -- Home Team mean mobile yellow cards received 	   
	   AVG(HY)
	   OVER( PARTITION BY HomeTeam
			 ORDER BY Game_Date 
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Home_Mean_YellowCards_Received,
       -- Away Team mean mobile yellow cards received 
	   AVG(AY)
	   OVER( PARTITION BY AwayTeam
			 ORDER BY Game_Date 
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Away_Mean_YellowCards_Received,
	   -- Home Team mean mobile red cards received 	   
	   AVG(HR)
	   OVER( PARTITION BY HomeTeam
			 ORDER BY Game_Date 
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Home_Mean_RedCards_Received,
       -- Away Team mean mobile red cards received 
	   AVG(AR)
	   OVER( PARTITION BY AwayTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Away_Mean_RedCards_Received,
		   
       -- Home Team mean mobile goals suffered
	   AVG(FTAG)
	   OVER( PARTITION BY HomeTeam
			 ORDER BY Game_Date 
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Home_Mean_Goals_Suffer,
       -- Away Team mean mobile goals suffered 
	   AVG(FTHG)
	   OVER( PARTITION BY AwayTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Away_Mean_Goals_Suffer,
       -- Home Team mean mobile shots on target suffered
	   AVG(AST)
	   OVER( PARTITION BY HomeTeam
			 ORDER BY Game_Date 
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Home_Mean_Shots_Target_Suffer,
       -- Away Team mean mobile shots on target suffered 
	   AVG(HST)
	   OVER( PARTITION BY AwayTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Away_Mean_Shots_Target_Suffer,
	   -- Home Team mean mobile goal suffered per shots suffered	   
	   AVG(AST/FTAG)
	   OVER( PARTITION BY HomeTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Home_Mean_Shots_per_Goals_Suffer,
       -- Away Team mean mobile suffered per shots suffered
	   AVG(HST/FTHG)
	   OVER( PARTITION BY AwayTeam
			 ORDER BY Game_Date 
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Away_Mean_Shots_per_Goals_Suffer,
	   -- Home Team mean mobile faults suffered 	   
	   AVG(AF)
	   OVER( PARTITION BY HomeTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Home_Mean_Faults_Suffer,
       -- Away Team mean mobile faults suffered
	   AVG(HF)
	   OVER( PARTITION BY AwayTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Away_Mean_Faults_Suffer,
	   -- Home Team mean mobile yellow cards opposite team 	   
	   AVG(AY)
	   OVER( PARTITION BY HomeTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Home_Mean_YellowCards_Opposite,
       -- Away Team mean mobile yellow cards opposite team 
	   AVG(HY)
	   OVER( PARTITION BY AwayTeam
			 ORDER BY Game_Date 
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Away_Mean_YellowCards_Opposite,
	   -- Home Team mean mobile red cards opposite team  	   
	   AVG(AR)
	   OVER( PARTITION BY HomeTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Home_Mean_RedCards_Opposite,
       -- Away Team mean mobile red cards opposite team  
	   AVG(HR)
	   OVER( PARTITION BY AwayTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
		   ) AS Away_Mean_RedCards_Opposite,
       -- Home Team mobile points gained 
 	   sum(CASE 
		 WHEN FTR = 'H' THEN 3
		 WHEN FTR = 'D' THEN 1
		 ELSE 0
	   END) 
		 OVER( PARTITION BY HomeTeam
		 ORDER BY Game_Date  
		 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
	   ) AS Home_Mobile_Points_Win,
       -- Away Team mobile points gained 
 	   sum(CASE 
		 WHEN FTR = 'A' THEN 3
		 WHEN FTR = 'D' THEN 1
		 ELSE 0
	   END) 
		 OVER( PARTITION BY AwayTeam
		 ORDER BY Game_Date 
		 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
	   ) AS Away_Mobile_Points_Win,
       -- Home Team mobile points lost 
 	   sum(CASE 
		 WHEN FTR = 'A' THEN 3
		 WHEN FTR = 'D' THEN 1
		 ELSE 0
	   END) 
		 OVER( PARTITION BY HomeTeam
		 ORDER BY Game_Date 
		 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
	   ) AS Home_Mobile_Points_Lost,
       -- Away Team mobile points lost
 	   sum(CASE 
		 WHEN FTR = 'H' THEN 3
		 WHEN FTR = 'D' THEN 1
		 ELSE 0
	   END) 
		 OVER( PARTITION BY AwayTeam
		 ORDER BY Game_Date 
		 ROWS BETWEEN 5 PRECEDING and 1 PRECEDING 
	   ) AS Away_Mobile_Points_Lost,
	   -- Points last game in Home
	   sum(CASE 
		 WHEN FTR = 'H' THEN 3
		 WHEN FTR = 'D' THEN 1
		 ELSE 0
	   END) 
		 OVER( PARTITION BY HomeTeam
		 ORDER BY Game_Date  
		 ROWS BETWEEN 1 PRECEDING and 1 PRECEDING 
	   ) AS Home_Mobile_Points_LastGame,
	   -- Points last game in Away
	   sum(CASE 
		 WHEN FTR = 'A' THEN 3
		 WHEN FTR = 'D' THEN 1
		 ELSE 0
	   END) 
		 OVER( PARTITION BY AwayTeam
		 ORDER BY Game_Date  
		 ROWS BETWEEN 1 PRECEDING and 1 PRECEDING 
	   ) AS Away_Mobile_Points_LastGame, 
	    -- Home Team win because opponent got red card
		Home_Win_Red_Against,
		-- Away Team win because opponent got red card
		Away_Win_Red_Against,	   
	   -- bet houses were chosen because both (IWH and WHH) exist in all tables from 2001 to 2019
       -- Home Team ratio of bet winning  / Away Team ratio of bet loosing
		Home_Mean_Bet,
       -- Draw ratio of bet 
		Draw_Mean_Bet,
        -- Home Team ratio of bet loosing / Away Team cratio of bet winning
		Away_Mean_Bet		   
From All_Results_Season_2nd
ORDER BY Game_Date

"""

df_mean_mob_res = pd.read_sql(query02, db_PremierLeague)
df_mean_mob_res.sample(2)

,Season,Game_Date,HomeTeam,AwayTeam,FTHG,FTAG,Result,HTHG,HTAG,HaltTime_Result,Home_Mean_Goals,Away_Mean_Goals,Home_Mean_HT_Goals,Away_Mean_HT_Goals,Home_Mean_Shots_Target,Away_Mean_Shots_Target,Home_Mean_Shots_per_Goals,Away_Mean_Shots_per_Goals,Home_Mean_Faults_Committed,Away_Mean_Faults_Committed,Home_Mean_YellowCards_Received,Away_Mean_YellowCards_Received,Home_Mean_RedCards_Received,Away_Mean_RedCards_Received,Home_Mean_Goals_Suffer,Away_Mean_Goals_Suffer,Home_Mean_Shots_Target_Suffer,Away_Mean_Shots_Target_Suffer,Home_Mean_Shots_per_Goals_Suffer,Away_Mean_Shots_per_Goals_Suffer,Home_Mean_Faults_Suffer,Away_Mean_Faults_Suffer,Home_Mean_YellowCards_Opposite,Away_Mean_YellowCards_Opposite,Home_Mean_RedCards_Opposite,Away_Mean_RedCards_Opposite,Home_Mobile_Points_Win,Away_Mobile_Points_Win,Home_Mobile_Points_Lost,Away_Mobile_Points_Lost,Home_Mobile_Points_LastGame,Away_Mobile_Points_LastGame,Home_Win_Red_Against,Away_Win_Red_Against,Home_Mean_Bet,Draw_Mean_Bet,Away_Mean_Bet
2586,Results_2007,2007-03-31,West Ham,Middlesbrough,2,0,H,2,0,H,1.4,0.8,0.6,0.2,7.0,4.2,5.75,3.666667,15.0,14.2,3.0,1.8,0.2,0.0,2.2,1.2,6.4,6.6,3.4,4.333333,15.8,11.2,2.4,0.8,0.0,0.0,1.0,5.0,13.0,8.0,0.0,1.0,0,0,0.254438,0.380671,0.364892
2669,Results_2008,2007-08-12,Man United,Reading,0,0,D,0,0,D,2.2,1.4,1.0,0.2,12.6,5.4,4.80,3.500000,10.8,10.6,1.8,0.6,0.0,0.0,0.8,1.4,3.0,8.4,3.0,7.000000,13.2,12.2,3.0,1.0,0.0,0.0,10.0,5.0,4.0,8.0,0.0,1.0,0,0,0.068060,0.286922,0.645018


# Query for Season results

>> The target of this query is to leverage the last 05 results of the team (home and away respectively) in the league season

In [716]:
# query mean mobile from 2001 until 2019 for past 05 games, leveraging the season results 

query03 = """

Select   
       -- Home Team mean mobile goals 
	   AVG(FTHG)
	   OVER( PARTITION BY Season, HomeTeam
			 ORDER BY Game_Date 
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING 
		   ) AS Home_Mean_Goals_Season,
       -- Away Team mean mobile goals 
	   AVG(FTAG)
	   OVER( PARTITION BY Season, AwayTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
		   ) AS Away_Mean_Goals_Season,
       -- Home Team mean mobile half time goals 
	   AVG(HTHG)
	   OVER( PARTITION BY Season, HomeTeam
			 ORDER BY Game_Date 
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
		   ) AS Home_Mean_HT_Goals_Season,
       -- Away Team mean mobile half time goals goals 
	   AVG(HTAG)
	   OVER( PARTITION BY Season, AwayTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
		   ) AS Away_Mean_HT_Goals_Season,
		   -- Home Team mean mobile shots on target 
	   AVG(HST)
	   OVER( PARTITION BY Season, HomeTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
		   ) AS Home_Mean_Shots_Target_Season,
       -- Away Team mean mobile shots on target 
	   AVG(AST)
	   OVER( PARTITION BY Season, AwayTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
		   ) AS Away_Mean_Shots_Target_Season,
	   -- Home Team mean mobile faults commited 	   
	   AVG(HF)
	   OVER( PARTITION BY Season, HomeTeam
			 ORDER BY Game_Date 
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
		   ) AS Home_Mean_Faults_Committed_Season,
       -- Away Team mean mobile faults commited
	   AVG(AF)
	   OVER( PARTITION BY Season, AwayTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
		   ) AS Away_Mean_Faults_Committed_Season,
	   -- Home Team mean mobile yellow cards received 	   
	   AVG(HY)
	   OVER( PARTITION BY Season, HomeTeam
			 ORDER BY Game_Date 
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING 
		   ) AS Home_Mean_YellowCards_Received_Season,
       -- Away Team mean mobile yellow cards received 
	   AVG(AY)
	   OVER( PARTITION BY Season, AwayTeam
			 ORDER BY Game_Date 
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
		   ) AS Away_Mean_YellowCards_Received_Season,
	   -- Home Team mean mobile red cards received 	   
	   AVG(HR)
	   OVER( PARTITION BY Season, HomeTeam
			 ORDER BY Game_Date 
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
		   ) AS Home_Mean_RedCards_Received_Season,
       -- Away Team mean mobile red cards received 
	   AVG(AR)
	   OVER( PARTITION BY Season, AwayTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
		   ) AS Away_Mean_RedCards_Received_Season,
		   
       -- Home Team mean mobile goals suffered
	   AVG(FTAG)
	   OVER( PARTITION BY Season, HomeTeam
			 ORDER BY Game_Date 
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
		   ) AS Home_Mean_Goals_Suffer_Season,
       -- Away Team mean mobile goals suffered 
	   AVG(FTHG)
	   OVER( PARTITION BY Season, AwayTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
		   ) AS Away_Mean_Goals_Suffer_Season,
       -- Home Team mean mobile shots on target suffered
	   AVG(AST)
	   OVER( PARTITION BY Season, HomeTeam
			 ORDER BY Game_Date 
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
		   ) AS Home_Mean_Shots_Target_Suffer_Season,
       -- Away Team mean mobile shots on target suffered 
	   AVG(HST)
	   OVER( PARTITION BY Season, AwayTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
		   ) AS Away_Mean_Shots_Target_Suffer_Season,
	   -- Home Team mean mobile goal suffered per shots suffered	   
 
	   AVG(AF)
	   OVER( PARTITION BY Season, HomeTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
		   ) AS Home_Mean_Faults_Suffer_Season,
       -- Away Team mean mobile faults suffered
	   AVG(HF)
	   OVER( PARTITION BY Season, AwayTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
		   ) AS Away_Mean_Faults_Suffer_Season,
	   -- Home Team mean mobile yellow cards opposite team 	   
	   AVG(AY)
	   OVER( PARTITION BY Season, HomeTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
		   ) AS Home_Mean_YellowCards_Opposite_Season,
       -- Away Team mean mobile yellow cards opposite team 
	   AVG(HY)
	   OVER( PARTITION BY Season, AwayTeam
			 ORDER BY Game_Date 
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
		   ) AS Away_Mean_YellowCards_Opposite_Season,
	   -- Home Team mean mobile red cards opposite team  	   
	   AVG(AR)
	   OVER( PARTITION BY Season, HomeTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING 
		   ) AS Home_Mean_RedCards_Opposite_Season,
       -- Away Team mean mobile red cards opposite team  
	   AVG(HR)
	   OVER( PARTITION BY Season, AwayTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
		   ) AS Away_Mean_RedCards_Opposite_Season,
       -- Home Team mobile points gained 
 	   sum(CASE 
		 WHEN FTR = 'H' THEN 3
		 WHEN FTR = 'D' THEN 1
		 ELSE 0
	   END) 
		 OVER( PARTITION BY Season, HomeTeam
		 ORDER BY Game_Date  
		 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
	   ) AS Home_Mobile_Points_Win_Season,
       -- Away Team mobile points gained 
 	   sum(CASE 
		 WHEN FTR = 'A' THEN 3
		 WHEN FTR = 'D' THEN 1
		 ELSE 0
	   END) 
		 OVER( PARTITION BY Season, AwayTeam
		 ORDER BY Game_Date 
		 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
	   ) AS Away_Mobile_Points_Win_Season,
       -- Home Team mobile points lost 
 	   sum(CASE 
		 WHEN FTR = 'A' THEN 3
		 WHEN FTR = 'D' THEN 1
		 ELSE 0
	   END) 
		 OVER( PARTITION BY Season, HomeTeam
		 ORDER BY Game_Date 
		 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
	   ) AS Home_Mobile_Points_Lost_Season,
       -- Away Team mobile points lost
 	   sum(CASE 
		 WHEN FTR = 'H' THEN 3
		 WHEN FTR = 'D' THEN 1
		 ELSE 0
	   END) 
		 OVER( PARTITION BY Season, AwayTeam
		 ORDER BY Game_Date 
		 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
	   ) AS Away_Mobile_Points_Lost_Season,
	 -- if Home Team won last 02 games
	 CASE WHEN (
 	   sum(CASE 
		 WHEN FTR = 'H' THEN 1
		 ELSE 0
	   END) 
		 OVER( PARTITION BY Season, HomeTeam
		 ORDER BY Game_Date  
		 ROWS BETWEEN 3 PRECEDING and 1 PRECEDING  
	   ) ) >= 2 THEN 1 ELSE 0 END AS Home_Mobile_Win_Season_Last02,
	-- if Away Team won last 02 games   
    CASE WHEN (
 	   sum(CASE 
		 WHEN FTR = 'A' THEN 1
		 ELSE 0
	   END) 
		 OVER( PARTITION BY Season, AwayTeam
		 ORDER BY Game_Date  
		 ROWS BETWEEN 3 PRECEDING and 1 PRECEDING  
	   ) ) >= 2 THEN 1 ELSE 0 END AS Away_Mobile_Win_Season_Last02,
	   -- Home Team total wins half and full times 
	   sum(Home_Win_Half_Full)
	   OVER( PARTITION BY Season, HomeTeam
			 ORDER BY Game_Date 
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING 
		   ) AS Home_Win_Half_Full,
       -- Away Team total wins half and full times 
	   sum(Away_Win_Half_Full)
	   OVER( PARTITION BY Season, AwayTeam
			 ORDER BY Game_Date  
			 ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING  
		   ) AS Away_Win_Half_Full
From All_Results_Season_2nd
ORDER BY Game_Date

"""

df_mean_mob_res_season = pd.read_sql(query03, db_PremierLeague)
df_mean_mob_res_season.sample(2)

,Home_Mean_Goals_Season,Away_Mean_Goals_Season,Home_Mean_HT_Goals_Season,Away_Mean_HT_Goals_Season,Home_Mean_Shots_Target_Season,Away_Mean_Shots_Target_Season,Home_Mean_Faults_Committed_Season,Away_Mean_Faults_Committed_Season,Home_Mean_YellowCards_Received_Season,Away_Mean_YellowCards_Received_Season,Home_Mean_RedCards_Received_Season,Away_Mean_RedCards_Received_Season,Home_Mean_Goals_Suffer_Season,Away_Mean_Goals_Suffer_Season,Home_Mean_Shots_Target_Suffer_Season,Away_Mean_Shots_Target_Suffer_Season,Home_Mean_Faults_Suffer_Season,Away_Mean_Faults_Suffer_Season,Home_Mean_YellowCards_Opposite_Season,Away_Mean_YellowCards_Opposite_Season,Home_Mean_RedCards_Opposite_Season,Away_Mean_RedCards_Opposite_Season,Home_Mobile_Points_Win_Season,Away_Mobile_Points_Win_Season,Home_Mobile_Points_Lost_Season,Away_Mobile_Points_Lost_Season,Home_Mobile_Win_Season_Last02,Away_Mobile_Win_Season_Last02,Home_Win_Half_Full,Away_Win_Half_Full
5492,2.5,1.875,1.125,0.75,7.125,5.125,10.625,12.625,1.375,1.875,0.0,0.0,0.375,1.5,1.375,4.375,10.5,10.625,1.625,1.5,0.25,0.125,24.0,12.0,0.0,9.0,1,0,6.0,2.0
6182,2.6,1.400,1.200,0.20,6.400,5.600,8.200,11.200,2.400,1.800,0.0,0.0,0.600,1.0,2.600,3.600,13.4,12.000,1.600,1.6,0.00,0.000,12.0,7.0,3.0,7.0,1,0,3.0,0.0


# DataSets concatenation - history and season league results

In [717]:
df_mean_results = pd.concat([df_mean_mob_res, df_mean_mob_res_season], axis=1)
df_mean_results.sample(2)

,Season,Game_Date,HomeTeam,AwayTeam,FTHG,FTAG,Result,HTHG,HTAG,HaltTime_Result,Home_Mean_Goals,Away_Mean_Goals,Home_Mean_HT_Goals,Away_Mean_HT_Goals,Home_Mean_Shots_Target,Away_Mean_Shots_Target,Home_Mean_Shots_per_Goals,Away_Mean_Shots_per_Goals,Home_Mean_Faults_Committed,Away_Mean_Faults_Committed,Home_Mean_YellowCards_Received,Away_Mean_YellowCards_Received,Home_Mean_RedCards_Received,Away_Mean_RedCards_Received,Home_Mean_Goals_Suffer,Away_Mean_Goals_Suffer,Home_Mean_Shots_Target_Suffer,Away_Mean_Shots_Target_Suffer,Home_Mean_Shots_per_Goals_Suffer,Away_Mean_Shots_per_Goals_Suffer,Home_Mean_Faults_Suffer,Away_Mean_Faults_Suffer,Home_Mean_YellowCards_Opposite,Away_Mean_YellowCards_Opposite,Home_Mean_RedCards_Opposite,Away_Mean_RedCards_Opposite,Home_Mobile_Points_Win,Away_Mobile_Points_Win,Home_Mobile_Points_Lost,Away_Mobile_Points_Lost,Home_Mobile_Points_LastGame,Away_Mobile_Points_LastGame,Home_Win_Red_Against,Away_Win_Red_Against,Home_Mean_Bet,Draw_Mean_Bet,Away_Mean_Bet,Home_Mean_Goals_Season,Away_Mean_Goals_Season,Home_Mean_HT_Goals_Season,Away_Mean_HT_Goals_Season,Home_Mean_Shots_Target_Season,Away_Mean_Shots_Target_Season,Home_Mean_Faults_Committed_Season,Away_Mean_Faults_Committed_Season,Home_Mean_YellowCards_Received_Season,Away_Mean_YellowCards_Received_Season,Home_Mean_RedCards_Received_Season,Away_Mean_RedCards_Received_Season,Home_Mean_Goals_Suffer_Season,Away_Mean_Goals_Suffer_Season,Home_Mean_Shots_Target_Suffer_Season,Away_Mean_Shots_Target_Suffer_Season,Home_Mean_Faults_Suffer_Season,Away_Mean_Faults_Suffer_Season,Home_Mean_YellowCards_Opposite_Season,Away_Mean_YellowCards_Opposite_Season,Home_Mean_RedCards_Opposite_Season,Away_Mean_RedCards_Opposite_Season,Home_Mobile_Points_Win_Season,Away_Mobile_Points_Win_Season,Home_Mobile_Points_Lost_Season,Away_Mobile_Points_Lost_Season,Home_Mobile_Win_Season_Last02,Away_Mobile_Win_Season_Last02,Home_Win_Half_Full,Away_Win_Half_Full
5012,Results_2014,2013-10-19,Everton,Hull,2,1,H,1,1,D,1.4,1.0,1.2,0.4,8.6,3.2,2.666667,2.333333,9.8,12.8,1.2,1.0,0.0,0.0,0.4,1.6,3.4,5.4,3.0,2.250000,13.8,11.0,2.2,1.0,0.0,0.0,13.0,5.0,1.0,8.0,3.0,3.0,0,0,0.108171,0.316394,0.575435,1.333333,1.000,1.333333,0.333333,6.666667,3.0000,9.666667,12.666667,1.333333,1.666667,0.000,0.0000,0.666667,2.000,4.333333,4.333333,14.0000,11.0,2.0000,1.0000,0.0000,0.0,7.0,3.0,1.0,6.0,1,0,2.0,0.0
4887,Results_2013,2013-04-20,Fulham,Arsenal,0,1,A,0,1,A,1.4,1.4,1.0,0.4,7.0,8.0,5.600000,4.000000,8.4,11.8,1.6,1.8,0.2,0.4,1.4,1.0,7.0,5.4,5.0,3.666667,10.2,8.4,1.2,2.2,0.0,0.0,9.0,9.0,6.0,6.0,0.0,3.0,0,0,0.456455,0.370484,0.173061,1.562500,1.375,0.875000,0.562500,7.250000,7.8125,9.875000,9.187500,1.125000,1.187500,0.125,0.1875,1.375000,0.875,7.625000,6.000000,10.9375,10.0,1.0625,2.1875,0.0625,0.0,24.0,26.0,21.0,17.0,1,1,6.0,4.0


## Filling missing values - Executed 2 times

In [729]:
# some data can be null, because teams can finish games without goals, cards and so on
# it could also be first games of seasons, thus that wouldn't show stats yet
# checking missing values

print(f'Missing values treino:\n{df_mean_results.isna().sum()}')

Missing values treino:
Season                                     0
Game_Date                                  0
HomeTeam                                   0
AwayTeam                                   0
FTHG                                       0
FTAG                                       0
Result                                     0
HTHG                                       0
HTAG                                       0
HaltTime_Result                            0
Home_Mean_Goals                            0
Away_Mean_Goals                            0
Home_Mean_HT_Goals                         0
Away_Mean_HT_Goals                         0
Home_Mean_Shots_Target                     0
Away_Mean_Shots_Target                     0
Home_Mean_Shots_per_Goals                  0
Away_Mean_Shots_per_Goals                  0
Home_Mean_Faults_Committed                 0
Away_Mean_Faults_Committed                 0
Home_Mean_YellowCards_Received             0
Away_Mean_YellowCards_Received  

In [730]:
# fill null values with zero, understanding that there aren't stats, for instace, it can first games of teams in seasons

df = df_mean_results.isna().sum()
for col in df_mean_results:
    if df[col] != 0:
        df_mean_results[col] = df_mean_results[col].fillna(value=0.0)

print(f'Missing values treino:\n{df_mean_results.isna().sum()}')

Missing values treino:
Season                                   0
Game_Date                                0
HomeTeam                                 0
AwayTeam                                 0
FTHG                                     0
FTAG                                     0
Result                                   0
HTHG                                     0
HTAG                                     0
HaltTime_Result                          0
Home_Mean_Goals                          0
Away_Mean_Goals                          0
Home_Mean_HT_Goals                       0
Away_Mean_HT_Goals                       0
Home_Mean_Shots_Target                   0
Away_Mean_Shots_Target                   0
Home_Mean_Shots_per_Goals                0
Away_Mean_Shots_per_Goals                0
Home_Mean_Faults_Committed               0
Away_Mean_Faults_Committed               0
Home_Mean_YellowCards_Received           0
Away_Mean_YellowCards_Received           0
Home_Mean_RedCards_Received    

## Feature engineering

In [720]:
# transform Result into numeric value
# Result will be the target variable
# H home winning means 0
# D draw means 1
# A away winning means 2
df_mean_results['Result'].loc[df_mean_results['Result'] == 'H'] = 0 
df_mean_results['Result'].loc[df_mean_results['Result'] == 'D'] = 1 
df_mean_results['Result'].loc[df_mean_results['Result'] == 'A'] = 2 

# transform Half time Result into numeric value
# H home winning means 0
# D draw means 1
# A away winning means 2
df_mean_results['HaltTime_Result'].loc[df_mean_results['HaltTime_Result'] == 'H'] = 0 
df_mean_results['HaltTime_Result'].loc[df_mean_results['HaltTime_Result'] == 'D'] = 1 
df_mean_results['HaltTime_Result'].loc[df_mean_results['HaltTime_Result'] == 'A'] = 2 

In [721]:
df_mean_results.loc[ (df_mean_results['Game_Date']=='2017-10-15') & (df_mean_results['HomeTeam']=='Brighton') ]

,Season,Game_Date,HomeTeam,AwayTeam,FTHG,FTAG,Result,HTHG,HTAG,HaltTime_Result,Home_Mean_Goals,Away_Mean_Goals,Home_Mean_HT_Goals,Away_Mean_HT_Goals,Home_Mean_Shots_Target,Away_Mean_Shots_Target,Home_Mean_Shots_per_Goals,Away_Mean_Shots_per_Goals,Home_Mean_Faults_Committed,Away_Mean_Faults_Committed,Home_Mean_YellowCards_Received,Away_Mean_YellowCards_Received,Home_Mean_RedCards_Received,Away_Mean_RedCards_Received,Home_Mean_Goals_Suffer,Away_Mean_Goals_Suffer,Home_Mean_Shots_Target_Suffer,Away_Mean_Shots_Target_Suffer,Home_Mean_Shots_per_Goals_Suffer,Away_Mean_Shots_per_Goals_Suffer,Home_Mean_Faults_Suffer,Away_Mean_Faults_Suffer,Home_Mean_YellowCards_Opposite,Away_Mean_YellowCards_Opposite,Home_Mean_RedCards_Opposite,Away_Mean_RedCards_Opposite,Home_Mobile_Points_Win,Away_Mobile_Points_Win,Home_Mobile_Points_Lost,Away_Mobile_Points_Lost,Home_Mobile_Points_LastGame,Away_Mobile_Points_LastGame,Home_Win_Red_Against,Away_Win_Red_Against,Home_Mean_Bet,Draw_Mean_Bet,Away_Mean_Bet,Home_Mean_Goals_Season,Away_Mean_Goals_Season,Home_Mean_HT_Goals_Season,Away_Mean_HT_Goals_Season,Home_Mean_Shots_Target_Season,Away_Mean_Shots_Target_Season,Home_Mean_Faults_Committed_Season,Away_Mean_Faults_Committed_Season,Home_Mean_YellowCards_Received_Season,Away_Mean_YellowCards_Received_Season,Home_Mean_RedCards_Received_Season,Away_Mean_RedCards_Received_Season,Home_Mean_Goals_Suffer_Season,Away_Mean_Goals_Suffer_Season,Home_Mean_Shots_Target_Suffer_Season,Away_Mean_Shots_Target_Suffer_Season,Home_Mean_Faults_Suffer_Season,Away_Mean_Faults_Suffer_Season,Home_Mean_YellowCards_Opposite_Season,Away_Mean_YellowCards_Opposite_Season,Home_Mean_RedCards_Opposite_Season,Away_Mean_RedCards_Opposite_Season,Home_Mobile_Points_Win_Season,Away_Mobile_Points_Win_Season,Home_Mobile_Points_Lost_Season,Away_Mobile_Points_Lost_Season,Home_Mobile_Win_Season_Last02,Away_Mobile_Win_Season_Last02,Home_Win_Half_Full,Away_Win_Half_Full
6537,Results_2018,2017-10-15,Brighton,Everton,1,1,1,0,0,1,1.333333,0.4,0.333333,0.2,3.666667,2.8,1.666667,2.5,7.0,11.6,1.333333,1.8,0.0,0.2,1.0,2.2,4.0,6.6,2.5,3.4,7.0,9.8,1.0,1.2,0.0,0.4,6.0,1.0,3.0,13.0,3.0,0.0,0,0,0.362504,0.359592,0.277904,1.333333,0.333333,0.333333,0.333333,3.666667,1.666667,7.0,10.0,1.333333,1.666667,0.0,0.333333,1.0,2.333333,4.0,6.666667,7.0,11.333333,1.0,1.333333,0.0,0.333333,6.0,1.0,3.0,7.0,1,0,1.0,0.0


In [120]:
# # round to 2 decimals
# def truncate(f):
#     '''Truncates/pads a float f to n decimal places without rounding'''
#     s = '{}'.format(f)
#     if 'e' in s or 'E' in s:
#         return '{0:.{1}f}'.format(f, 2)
#     i, p, d = s.partition('.')
#     return '.'.join([i, (d+'0'*2)[:2]])

In [248]:
# vars_float = df_mean_results.select_dtypes(include=['float64'])

# for col in vars_float:
#     df_mean_results[col] = df_mean_results[col].apply(truncate)

In [264]:
# transform Result and half time Result into numbering
df_mean_results['Result'] = df_mean_results['Result'].astype('int64')
df_mean_results['HaltTime_Result'] = df_mean_results['HaltTime_Result'].astype('int64')

In [722]:
# function history of team results in the league - wins, draw and losts of the game
# for example, what are the result history of Arsenal vs Chelsea

def Result_History(field, Result):

    df_mean_results[field] = 0
    
    for row in range(df_mean_results.shape[0]):
        HomeTeam = df_mean_results['HomeTeam'][row]
        AwayTeam = df_mean_results['AwayTeam'][row]
        GameDate = df_mean_results['Game_Date'][row]
        GameDate = GameDate[0:4] + GameDate[5:7] + GameDate [-2:]
        GameDate = int(GameDate) 
        GameDate = GameDate - 1
        GameDate = str(GameDate)

        query04 = "select count(FTR) as FTR from All_Results_Season where HomeTeam = '" + HomeTeam + "' and AwayTeam = '" + AwayTeam + "' and CAST('" + GameDate + "' as INTEGER) > CAST(substr(Game_Date, 1, 4) ||  substr(Game_Date, 6, 2) || substr(Game_Date, 9, 2) as INTEGER) and FTR = '" + Result +  "'"

        df_hist_result = pd.read_sql_query(query04, db_PremierLeague)

        df_mean_results[field][row] = df_hist_result['FTR']

In [723]:
# including the history results per game
Result_History('wins_Home', 'H')
Result_History('wins_Away', 'A')
Result_History('draws_Hist', 'D')

In [724]:
df_mean_results.sample(3)

,Season,Game_Date,HomeTeam,AwayTeam,FTHG,FTAG,Result,HTHG,HTAG,HaltTime_Result,Home_Mean_Goals,Away_Mean_Goals,Home_Mean_HT_Goals,Away_Mean_HT_Goals,Home_Mean_Shots_Target,Away_Mean_Shots_Target,Home_Mean_Shots_per_Goals,Away_Mean_Shots_per_Goals,Home_Mean_Faults_Committed,Away_Mean_Faults_Committed,Home_Mean_YellowCards_Received,Away_Mean_YellowCards_Received,Home_Mean_RedCards_Received,Away_Mean_RedCards_Received,Home_Mean_Goals_Suffer,Away_Mean_Goals_Suffer,Home_Mean_Shots_Target_Suffer,Away_Mean_Shots_Target_Suffer,Home_Mean_Shots_per_Goals_Suffer,Away_Mean_Shots_per_Goals_Suffer,Home_Mean_Faults_Suffer,Away_Mean_Faults_Suffer,Home_Mean_YellowCards_Opposite,Away_Mean_YellowCards_Opposite,Home_Mean_RedCards_Opposite,Away_Mean_RedCards_Opposite,Home_Mobile_Points_Win,Away_Mobile_Points_Win,Home_Mobile_Points_Lost,Away_Mobile_Points_Lost,Home_Mobile_Points_LastGame,Away_Mobile_Points_LastGame,Home_Win_Red_Against,Away_Win_Red_Against,Home_Mean_Bet,Draw_Mean_Bet,Away_Mean_Bet,Home_Mean_Goals_Season,Away_Mean_Goals_Season,Home_Mean_HT_Goals_Season,Away_Mean_HT_Goals_Season,Home_Mean_Shots_Target_Season,Away_Mean_Shots_Target_Season,Home_Mean_Faults_Committed_Season,Away_Mean_Faults_Committed_Season,Home_Mean_YellowCards_Received_Season,Away_Mean_YellowCards_Received_Season,Home_Mean_RedCards_Received_Season,Away_Mean_RedCards_Received_Season,Home_Mean_Goals_Suffer_Season,Away_Mean_Goals_Suffer_Season,Home_Mean_Shots_Target_Suffer_Season,Away_Mean_Shots_Target_Suffer_Season,Home_Mean_Faults_Suffer_Season,Away_Mean_Faults_Suffer_Season,Home_Mean_YellowCards_Opposite_Season,Away_Mean_YellowCards_Opposite_Season,Home_Mean_RedCards_Opposite_Season,Away_Mean_RedCards_Opposite_Season,Home_Mobile_Points_Win_Season,Away_Mobile_Points_Win_Season,Home_Mobile_Points_Lost_Season,Away_Mobile_Points_Lost_Season,Home_Mobile_Win_Season_Last02,Away_Mobile_Win_Season_Last02,Home_Win_Half_Full,Away_Win_Half_Full,wins_Home,wins_Away,draws_Hist
3805,Results_2011,2010-08-14,Tottenham,Man City,0,0,1,0,0,1,2.0,2.0,1.4,1.6,9.4,8.0,7.0,3.75,9.6,9.6,1.0,1.2,0.0,0.0,0.6,0.8,6.8,6.0,8.0,6.500000,10.0,10.0,1.8,1.6,0.2,0.0,15.0,9.0,0.0,3.0,3.0,1.0,0,0,0.276896,0.383108,0.339996,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0,0,0.0,0.0,6,1,2
5447,Results_2015,2014-11-29,West Ham,Newcastle,1,0,0,0,0,1,1.6,1.2,1.0,0.4,4.2,2.8,2.4,3.25,11.6,11.0,2.0,2.2,0.0,0.0,1.0,1.6,5.0,3.8,4.0,2.000000,11.8,13.4,1.6,0.8,0.0,0.0,10.0,7.0,4.0,7.0,1.0,3.0,1,0,0.241646,0.374369,0.383985,1.333333,1.000000,0.833333,0.333333,4.166667,3.000000,11.666667,10.500000,1.833333,1.833333,0.166667,0.166667,1.000000,1.333333,4.833333,3.166667,11.500000,13.000000,1.333333,1.333333,0.166667,0.000000,10.0,8.0,7.0,8.0,1,1,3.0,1.0,3,4,3
239,Results_2001,2001-01-31,Everton,Middlesbrough,2,2,1,0,1,2,1.2,0.2,0.0,0.2,5.8,2.6,3.2,2.00,13.2,15.6,1.4,2.8,0.0,0.2,0.8,0.6,3.6,6.2,4.0,5.333333,13.6,16.8,1.4,1.4,0.2,0.0,8.0,3.0,5.0,9.0,1.0,1.0,0,0,0.284483,0.395320,0.320197,1.181818,1.083333,0.181818,0.500000,5.636364,3.583333,13.000000,15.666667,1.818182,2.500000,0.000000,0.083333,1.272727,1.166667,4.272727,6.250000,13.090909,12.666667,1.545455,1.750000,0.181818,0.083333,13.0,11.0,16.0,20.0,0,0,0.0,1.0,0,0,0


In [725]:
df_mean_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7220 entries, 0 to 7219
Data columns (total 80 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Season                                 7220 non-null   object 
 1   Game_Date                              7220 non-null   object 
 2   HomeTeam                               7220 non-null   object 
 3   AwayTeam                               7220 non-null   object 
 4   FTHG                                   7220 non-null   int64  
 5   FTAG                                   7220 non-null   int64  
 6   Result                                 7220 non-null   object 
 7   HTHG                                   7220 non-null   int64  
 8   HTAG                                   7220 non-null   int64  
 9   HaltTime_Result                        7220 non-null   object 
 10  Home_Mean_Goals                        7220 non-null   float64
 11  Away

In [726]:
# including team season results, regardless home or away - number of points, usage percentage, and if the team scored in the 
# last 03 games sequentially

df_mean_results['Home_Point_per_Season'] = 0
df_mean_results['Home_Perc_per_Season'] = 0
df_mean_results['Home_Score_Last_3Games'] = 0

df_mean_results['Away_Point_per_Season'] = 0
df_mean_results['Away_Perc_per_Season'] = 0
df_mean_results['Away_Score_Last_3Games'] = 0


Seasons = ['Results_2001', 'Results_2002', 'Results_2003', 'Results_2004', 'Results_2005', 'Results_2006', 'Results_2007',
'Results_2008', 'Results_2009', 'Results_2010', 'Results_2011', 'Results_2012', 'Results_2013', 'Results_2014', 'Results_2015',
'Results_2016', 'Results_2017', 'Results_2018', 'Results_2019']

for i in range(len(Seasons)):
    
    query04 = """
                select DISTINCT HomeTeam from All_Results_Season_2nd WHERE Season = '""" + Seasons[i] + """'
              """
    df_Teams = pd.read_sql(query04, db_PremierLeague)
    array_Teams = np.array(df_Teams['HomeTeam'])

    for y in range(array_Teams.shape[0]):
        
        query05 = """

                    SELECT Season, Game_Date, HomeTeam as Team, FTR, 'Home' as 'Local', FTHG as 'Goals',  
                    CASE WHEN FTR = 'H' THEN 3
                         WHEN FTR = 'D' THEN 1
                         ELSE 0
                    END as Point_Game
                    from All_Results_Season_2nd
                    where HomeTeam = '""" + array_Teams[y] + """' and Season = '""" + Seasons[i] + """'
                    UNION
                    SELECT Season, Game_Date, AwayTeam as Team, FTR, 'Away' as 'Local', FTAG as 'Goals', 
                    CASE WHEN FTR = 'A' THEN 3
                         WHEN FTR = 'D' THEN 1
                         ELSE 0
                    END as Point_Game
                    from All_Results_Season_2nd
                    where AwayTeam =  '""" + array_Teams[y] + """' and Season = '""" + Seasons[i] + """'
                    ORDER BY Game_Date

                  """
        # all results of one team in the season
        df = pd.read_sql(query05, db_PremierLeague)
        
        # number of points until the last game
        df['Point_per_Season'] = df['Point_Game'].rolling(38, min_periods=1).sum().shift(periods=1)
        
        # ratio of points conquested until the last game
        df['Perc_per_Season'] = df['Point_per_Season'] / df['Point_Game'].\
        rolling(38, min_periods=1).count().shift(periods=1).apply(lambda x: x*3)
        
        # if team scored goal in the last game
        df['Goal_Last_Game'] = df['Goals'].rolling(1, min_periods=1).sum().shift(periods=1)>0
        df['Goal_Last_Game'].loc[ (df['Goal_Last_Game'] == True) ] = 1
        df['Goal_Last_Game'].loc[ (df['Goal_Last_Game'] == False) ] = 0
        
        # if past 3 games consecutive games team scored goals 
        df['Match_Last_3Games'] = 0
        df['Match_Last_3Games'].loc[df['Goal_Last_Game'].rolling(3, min_periods=1).sum().shift(periods=1)==3]=1    

        # inclusion in the dataset Point_per_Season, Perc_per_Season, Match_Last_3Games

        for n in range(df.shape[0]):
            if df['Local'][n] == 'Home':
                df_mean_results['Home_Point_per_Season'].loc[ (df_mean_results['Season'] == df['Season'][n]) &\
                                                             (df_mean_results['HomeTeam'] == df['Team'][n]) &\
                                                             (df_mean_results['Game_Date'] == df['Game_Date'][n]) ] \
                = df['Point_per_Season'][n]

                df_mean_results['Home_Perc_per_Season'].loc[ (df_mean_results['Season'] == df['Season'][n]) &\
                                                             (df_mean_results['HomeTeam'] == df['Team'][n]) &\
                                                             (df_mean_results['Game_Date'] == df['Game_Date'][n]) ] \
                = df['Perc_per_Season'][n]
                
                df_mean_results['Home_Score_Last_3Games'].loc[ (df_mean_results['Season'] == df['Season'][n]) &\
                                                             (df_mean_results['HomeTeam'] == df['Team'][n]) &\
                                                             (df_mean_results['Game_Date'] == df['Game_Date'][n]) ] \
                = df['Match_Last_3Games'][n]
            
            else:
                df_mean_results['Away_Point_per_Season'].loc[ (df_mean_results['Season'] == df['Season'][n]) &\
                                                             (df_mean_results['AwayTeam'] == df['Team'][n]) &\
                                                             (df_mean_results['Game_Date'] == df['Game_Date'][n]) ] \
                = df['Point_per_Season'][n]

                df_mean_results['Away_Perc_per_Season'].loc[ (df_mean_results['Season'] == df['Season'][n]) &\
                                                             (df_mean_results['AwayTeam'] == df['Team'][n]) &\
                                                             (df_mean_results['Game_Date'] == df['Game_Date'][n]) ] \
                = df['Perc_per_Season'][n]
                
                df_mean_results['Away_Score_Last_3Games'].loc[ (df_mean_results['Season'] == df['Season'][n]) &\
                                                             (df_mean_results['AwayTeam'] == df['Team'][n]) &\
                                                             (df_mean_results['Game_Date'] == df['Game_Date'][n]) ] \
                = df['Match_Last_3Games'][n]

In [728]:
df_mean_results.sample(6)

,Season,Game_Date,HomeTeam,AwayTeam,FTHG,FTAG,Result,HTHG,HTAG,HaltTime_Result,Home_Mean_Goals,Away_Mean_Goals,Home_Mean_HT_Goals,Away_Mean_HT_Goals,Home_Mean_Shots_Target,Away_Mean_Shots_Target,Home_Mean_Shots_per_Goals,Away_Mean_Shots_per_Goals,Home_Mean_Faults_Committed,Away_Mean_Faults_Committed,Home_Mean_YellowCards_Received,Away_Mean_YellowCards_Received,Home_Mean_RedCards_Received,Away_Mean_RedCards_Received,Home_Mean_Goals_Suffer,Away_Mean_Goals_Suffer,Home_Mean_Shots_Target_Suffer,Away_Mean_Shots_Target_Suffer,Home_Mean_Shots_per_Goals_Suffer,Away_Mean_Shots_per_Goals_Suffer,Home_Mean_Faults_Suffer,Away_Mean_Faults_Suffer,Home_Mean_YellowCards_Opposite,Away_Mean_YellowCards_Opposite,Home_Mean_RedCards_Opposite,Away_Mean_RedCards_Opposite,Home_Mobile_Points_Win,Away_Mobile_Points_Win,Home_Mobile_Points_Lost,Away_Mobile_Points_Lost,Home_Mobile_Points_LastGame,Away_Mobile_Points_LastGame,Home_Win_Red_Against,Away_Win_Red_Against,Home_Mean_Bet,Draw_Mean_Bet,Away_Mean_Bet,Home_Mean_Goals_Season,Away_Mean_Goals_Season,Home_Mean_HT_Goals_Season,Away_Mean_HT_Goals_Season,Home_Mean_Shots_Target_Season,Away_Mean_Shots_Target_Season,Home_Mean_Faults_Committed_Season,Away_Mean_Faults_Committed_Season,Home_Mean_YellowCards_Received_Season,Away_Mean_YellowCards_Received_Season,Home_Mean_RedCards_Received_Season,Away_Mean_RedCards_Received_Season,Home_Mean_Goals_Suffer_Season,Away_Mean_Goals_Suffer_Season,Home_Mean_Shots_Target_Suffer_Season,Away_Mean_Shots_Target_Suffer_Season,Home_Mean_Faults_Suffer_Season,Away_Mean_Faults_Suffer_Season,Home_Mean_YellowCards_Opposite_Season,Away_Mean_YellowCards_Opposite_Season,Home_Mean_RedCards_Opposite_Season,Away_Mean_RedCards_Opposite_Season,Home_Mobile_Points_Win_Season,Away_Mobile_Points_Win_Season,Home_Mobile_Points_Lost_Season,Away_Mobile_Points_Lost_Season,Home_Mobile_Win_Season_Last02,Away_Mobile_Win_Season_Last02,Home_Win_Half_Full,Away_Win_Half_Full,wins_Home,wins_Away,draws_Hist,Home_Point_per_Season,Home_Perc_per_Season,Home_Score_Last_3Games,Away_Point_per_Season,Away_Perc_per_Season,Away_Score_Last_3Games
6257,Results_2017,2016-12-26,Watford,Crystal Palace,1,1,1,0,1,2,1.60,2.2,0.60,0.2,3.6,4.8,1.750000,2.000000,15.4,13.8,2.20,3.0,0.2,0.0,1.2,3.0,2.2,5.8,1.500000,1.800000,13.60,13.0,2.2,2.4,0.0,0.0,10.0,2.0,4.0,11.0,3.0,1.0,0,0,0.277895,0.373883,0.348222,1.625000,2.000,0.500000,0.250,3.875000,4.25,16.125000,12.375,3.000000,2.750,0.125,0.000,1.500000,2.375,3.000000,5.250,14.000000,13.00,2.375,2.00,0.000,0.00,13.0,8.0,10.0,14.0,1,0,2.0,0.0,0,1,0,21.0,0.411765,0,15.0,0.294118,1
3902,Results_2011,2010-11-06,Blackpool,Everton,2,2,1,1,1,1,1.75,0.6,0.25,0.2,6.5,7.4,6.000000,4.250000,12.5,12.4,1.25,0.6,0.0,0.0,2.0,0.6,7.0,4.2,3.500000,5.333333,12.25,14.6,1.5,2.2,0.5,0.0,4.0,5.0,7.0,8.0,3.0,1.0,0,0,0.450286,0.362658,0.187055,1.750000,0.600,0.250000,0.200,6.500000,7.40,12.500000,12.400,1.250000,0.600,0.000,0.000,2.000000,0.600,7.000000,4.200,12.250000,14.60,1.500,2.20,0.500,0.00,4.0,5.0,7.0,8.0,0,0,1.0,0.0,0,0,0,13.0,0.433333,0,13.0,0.433333,1
5377,Results_2015,2014-09-27,Sunderland,Swansea,0,0,1,0,0,1,2.00,1.8,1.20,1.0,3.8,2.8,2.000000,3.250000,11.6,14.6,2.40,2.4,0.0,0.0,1.2,1.6,3.4,5.2,2.333333,3.800000,13.20,10.4,1.8,1.2,0.2,0.0,8.0,9.0,5.0,6.0,1.0,0.0,0,0,0.322390,0.368352,0.309258,1.500000,2.000,1.000000,1.000,2.000000,3.00,10.000000,15.500,2.500000,3.500,0.000,0.000,1.500000,2.500,4.500000,7.000,14.000000,13.00,1.500,1.00,0.000,0.00,2.0,3.0,2.0,3.0,0,0,0.0,1.0,1,1,1,4.0,0.266667,0,9.0,0.600000,1
4762,Results_2013,2013-01-01,West Brom,Fulham,1,2,2,0,1,2,1.20,0.8,0.60,0.4,8.2,5.2,3.750000,2.000000,9.4,10.4,1.00,1.4,0.0,0.0,0.6,2.0,5.4,9.2,6.666667,3.250000,11.60,11.8,1.4,1.4,0.0,0.0,10.0,2.0,4.0,11.0,3.0,0.0,0,0,0.215720,0.372607,0.411674,1.600000,1.300,0.600000,0.400,7.900000,7.20,10.900000,9.900,1.400000,1.200,0.000,0.000,0.700000,2.200,5.900000,9.300,11.700000,10.90,2.000,1.20,0.200,0.00,22.0,7.0,7.0,19.0,0,0,3.0,1.0,3,0,3,33.0,0.550000,0,21.0,0.350000,0
554,Results_2002,2001-12-22,Su

# Feature Selection

In [731]:
# define Target and drop variables
TARGET = 'Result'
vars_drop = ['Season', 'Game_Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'HTHG', 'HTAG', 'HaltTime_Result', 'Result']

In [745]:
# Dividing variables
df_mean_results[TARGET] = df_mean_results[TARGET].astype('int64')
X = df_mean_results.drop(vars_drop, axis=1)
Y = df_mean_results[TARGET]

In [746]:
# Defining categorical and numeric variables
vars_cat = ['Home_Win_Red_Against', 'Away_Win_Red_Against', 'Home_Mobile_Win_Season_Last02', 'Away_Mobile_Win_Season_Last02', \
            'Home_Score_Last_3Games', 'Away_Score_Last_3Games']
vars_num = X.select_dtypes(include=['int64', 'float64'])
vars_num = vars_num.drop(vars_cat, axis=1)

vars_cat = X.loc[:, vars_cat]

# ANOVA Validation

In [748]:
# ANOVA análise
selected_anova = f_classif(vars_num, Y)
print(selected_anova)

p_values_num = pd.Series(selected_anova[1])
p_values_num.index = vars_num.columns 
p_values_num = p_values_num[p_values_num<0.05]
p_values_num.index

(array([9.20373905e+01, 1.08337305e+02, 5.64612093e+01, 6.25515189e+01,
       9.85861649e+01, 9.51686593e+01, 1.60773284e+00, 3.91429592e+00,
       7.50313051e+00, 7.87333730e+00, 1.27038476e+01, 9.84599860e-01,
       7.03486825e+00, 1.32265812e+00, 6.58500054e+01, 6.59293901e+01,
       5.35615651e+01, 7.33687336e+01, 4.24452900e+00, 5.90073275e+00,
       8.09428854e-01, 1.11072762e+01, 1.28787398e+01, 4.87779326e+01,
       1.05330591e+00, 2.48877246e+00, 1.04674202e+02, 1.35767064e+02,
       1.01900260e+02, 1.03287338e+02, 2.93215213e+01, 3.84959221e+01,
       7.61438022e+02, 1.32253060e+02, 7.46784177e+02, 8.59699710e+01,
       8.61881384e+01, 4.32144842e+01, 4.65736262e+01, 8.25543607e+01,
       6.58915660e+01, 9.27953155e-01, 7.56455190e+00, 5.17597487e+00,
       9.12740949e-02, 1.56159659e+00, 1.60079639e-01, 4.85236700e+01,
       6.25675064e+01, 2.84582032e+01, 6.16833564e+01, 1.11697192e+00,
       2.34863049e+00, 1.02115833e+01, 1.81740411e+01, 4.30386298e+00,
     

Index(['Home_Mean_Goals', 'Away_Mean_Goals', 'Home_Mean_HT_Goals',
       'Away_Mean_HT_Goals', 'Home_Mean_Shots_Target',
       'Away_Mean_Shots_Target', 'Away_Mean_Shots_per_Goals',
       'Home_Mean_Faults_Committed', 'Away_Mean_Faults_Committed',
       'Home_Mean_YellowCards_Received', 'Home_Mean_RedCards_Received',
       'Home_Mean_Goals_Suffer', 'Away_Mean_Goals_Suffer',
       'Home_Mean_Shots_Target_Suffer', 'Away_Mean_Shots_Target_Suffer',
       'Home_Mean_Shots_per_Goals_Suffer', 'Away_Mean_Shots_per_Goals_Suffer',
       'Away_Mean_Faults_Suffer', 'Home_Mean_YellowCards_Opposite',
       'Away_Mean_YellowCards_Opposite', 'Home_Mobile_Points_Win',
       'Away_Mobile_Points_Win', 'Home_Mobile_Points_Lost',
       'Away_Mobile_Points_Lost', 'Home_Mobile_Points_LastGame',
       'Away_Mobile_Points_LastGame', 'Home_Mean_Bet', 'Draw_Mean_Bet',
       'Away_Mean_Bet', 'Home_Mean_Goals_Season', 'Away_Mean_Goals_Season',
       'Home_Mean_HT_Goals_Season', 'Away_Mean_HT_Goals_Se

# CHI2 Validation

In [749]:
# chi2 análise
selected_chi2 = chi2(vars_cat, Y)
selected_chi2

(array([420.01638368, 479.58402744,  56.43198641, 105.40371695,
         30.34925246,  37.20531015]),
 array([6.23162241e-092, 7.23855098e-105, 5.57121413e-013, 1.29381946e-023,
        2.56887850e-007, 8.33622737e-009]))

In [750]:
p_values_cat = pd.Series(selected_chi2[1])
p_values_cat.index = vars_cat.columns 
p_values_cat = p_values_cat[p_values_cat<0.05]
p_values_cat.index

Index(['Home_Win_Red_Against', 'Away_Win_Red_Against',
       'Home_Mobile_Win_Season_Last02', 'Away_Mobile_Win_Season_Last02',
       'Home_Score_Last_3Games', 'Away_Score_Last_3Games'],
      dtype='object')

# Boruta Evaluation

In [753]:
from sklearn.ensemble import RandomForestRegressor
# Random Forest Regressor for Boruta - features importance
forest = RandomForestRegressor(n_jobs = -1, max_depth = 3)

# fit boruta
boruta_selector = BorutaPy(forest, n_estimators = 30, max_iter=50, random_state = 42)
boruta_selector.fit(np.array(vars_num), np.array(Y))

BorutaPy(estimator=RandomForestRegressor(max_depth=3, n_estimators=30,
                                         n_jobs=-1,
                                         random_state=RandomState(MT19937) at 0x16A441E8940),
         max_iter=50, n_estimators=30,
         random_state=RandomState(MT19937) at 0x16A441E8940)

In [754]:
# numeric variables
boruta_selector.fit(np.array(vars_num), np.array(Y))

BorutaPy(estimator=RandomForestRegressor(max_depth=3, n_estimators=30,
                                         n_jobs=-1,
                                         random_state=RandomState(MT19937) at 0x16A441E8940),
         max_iter=50, n_estimators=30,
         random_state=RandomState(MT19937) at 0x16A441E8940)

In [755]:
# columnas consideradas pelo boruto
vars_num.loc[:, boruta_selector.support_].columns.tolist()

['Home_Mean_Bet', 'Away_Mean_Bet']

In [761]:
# categorical variables
boruta_selector.fit(np.array(vars_cat), np.array(Y))

BorutaPy(estimator=RandomForestRegressor(max_depth=3, n_estimators=30,
                                         n_jobs=-1,
                                         random_state=RandomState(MT19937) at 0x16A441E8940),
         max_iter=50, n_estimators=30,
         random_state=RandomState(MT19937) at 0x16A441E8940)

In [762]:
# categorical variables estimated by Boruta
vars_cat.loc[:, boruta_selector.support_].columns.tolist()

['Home_Win_Red_Against',
 'Away_Win_Red_Against',
 'Home_Mobile_Win_Season_Last02',
 'Away_Mobile_Win_Season_Last02']

# REFCV evaluation

In [756]:
# RFE - var_num_non_direct

forest = RandomForestRegressor(n_jobs = -1, max_depth = 3)

# 1a Selação (Random Forest) - neg_mean_squared_error: busca inverter a queda de performance, aumentou o MSE piora 
# então negativa pra melhorar 
rfecv_RFC = RFECV(estimator=forest, scoring='neg_mean_squared_error')
rfecv_RFC.fit(np.array(vars_num), np.array(Y))

RFECV(estimator=RandomForestRegressor(max_depth=3, n_jobs=-1),
      scoring='neg_mean_squared_error')

In [757]:
print(rfecv_RFC.n_features_) # Número de features selecionadas como as mais importantes 
print(rfecv_RFC.support_) # Vetor de booleans. True se a variável deve ser mantida e False caso contrário
mask_RFC = rfecv_RFC.support_
cols_drop_RFE= [ x for x in mask_RFC] # apenas invertendo o vetor de true/false
cols_drop_RFE= vars_num.loc[:,cols_drop_RFE].columns.tolist()
cols_drop_RFE

1
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False  True False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False]


['Home_Mean_Bet']

In [763]:
# categorical variables
rfecv_RFC.fit(np.array(vars_cat), np.array(Y))

RFECV(estimator=RandomForestRegressor(max_depth=3, n_jobs=-1),
      scoring='neg_mean_squared_error')

In [764]:
# categorical variables estimated by RFECV
print(rfecv_RFC.n_features_) # Número de features selecionadas como as mais importantes 
print(rfecv_RFC.support_) # Vetor de booleans. True se a variável deve ser mantida e False caso contrário
mask_RFC = rfecv_RFC.support_
cols_drop_RFE= [ x for x in mask_RFC] # apenas invertendo o vetor de true/false
cols_drop_RFE= vars_cat.loc[:,cols_drop_RFE].columns.tolist()
cols_drop_RFE

4
[ True  True  True  True False False]


['Home_Win_Red_Against',
 'Away_Win_Red_Against',
 'Home_Mobile_Win_Season_Last02',
 'Away_Mobile_Win_Season_Last02']

# Dividing train and test / Filling pipelines

In [765]:
# dividing train and test
df_train, df_test = train_test_split(df_mean_results, stratify=df_mean_results[TARGET] , test_size=0.2, random_state=42)

In [766]:
# charging data train and test
X_train = df_train.drop(TARGET, axis=1)
y_train = df_train[TARGET]

X_test = df_test.drop(TARGET, axis=1)
y_test = df_test[TARGET]

In [767]:
# chosen variables to pipe - complete
pipe_num_min_max = ('min_max_scaler', MinMaxScaler(), ['Home_Mean_Goals', 'Away_Mean_Goals', 'Home_Mean_HT_Goals',
       'Away_Mean_HT_Goals', 'Home_Mean_Shots_Target',
       'Away_Mean_Shots_Target', 'Away_Mean_Shots_per_Goals',
       'Home_Mean_Faults_Committed', 'Away_Mean_Faults_Committed',
       'Home_Mean_YellowCards_Received', 'Home_Mean_RedCards_Received',
       'Home_Mean_Goals_Suffer', 'Away_Mean_Goals_Suffer',
       'Home_Mean_Shots_Target_Suffer', 'Away_Mean_Shots_Target_Suffer',
       'Home_Mean_Shots_per_Goals_Suffer', 'Away_Mean_Shots_per_Goals_Suffer',
       'Away_Mean_Faults_Suffer', 'Home_Mean_YellowCards_Opposite',
       'Away_Mean_YellowCards_Opposite', 'Home_Mobile_Points_Win',
       'Away_Mobile_Points_Win', 'Home_Mobile_Points_Lost',
       'Away_Mobile_Points_Lost', 'Home_Mobile_Points_LastGame',
       'Away_Mobile_Points_LastGame', 'Home_Mean_Bet', 'Draw_Mean_Bet',
       'Away_Mean_Bet', 'Home_Mean_Goals_Season', 'Away_Mean_Goals_Season',
       'Home_Mean_HT_Goals_Season', 'Away_Mean_HT_Goals_Season',
       'Home_Mean_Shots_Target_Season', 'Away_Mean_Shots_Target_Season',
       'Away_Mean_Faults_Committed_Season',
       'Home_Mean_YellowCards_Received_Season',
       'Home_Mean_Goals_Suffer_Season', 'Away_Mean_Goals_Suffer_Season',
       'Home_Mean_Shots_Target_Suffer_Season',
       'Away_Mean_Shots_Target_Suffer_Season',
       'Home_Mean_YellowCards_Opposite_Season',
       'Away_Mean_YellowCards_Opposite_Season',
       'Home_Mean_RedCards_Opposite_Season',
       'Away_Mean_RedCards_Opposite_Season', 'Home_Mobile_Points_Win_Season',
       'Away_Mobile_Points_Win_Season', 'Home_Mobile_Points_Lost_Season',
       'Away_Mobile_Points_Lost_Season', 'Home_Win_Half_Full',
       'Away_Win_Half_Full', 'wins_Home', 'wins_Away', 'draws_Hist',
       'Home_Point_per_Season', 'Home_Perc_per_Season',
       'Away_Point_per_Season', 'Away_Perc_per_Season'
])

pipe_cat_one_hot = ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'), \
                    ['Home_Win_Red_Against', 'Away_Win_Red_Against', 'Home_Mobile_Win_Season_Last02', 
                     'Away_Mobile_Win_Season_Last02', 'Home_Score_Last_3Games', 'Away_Score_Last_3Games'])


transformers = [pipe_num_min_max, pipe_cat_one_hot]
pre_processador = ColumnTransformer(transformers)

In [768]:
# chosen variables to pipe only selected by Boruta and RFECV 
pipe_num_min_max_BR = ('min_max_scaler', MinMaxScaler(), ['Home_Mean_Bet', 
       'Away_Mean_Bet'])

pipe_cat_one_hot_BR = ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'), \
                    ['Home_Win_Red_Against', 'Away_Win_Red_Against', 'Home_Mobile_Win_Season_Last02', 
                     'Away_Mobile_Win_Season_Last02'])


transformers_BR = [pipe_num_min_max_BR, pipe_cat_one_hot_BR]
pre_processador_BR = ColumnTransformer(transformers_BR)

## Logistic Regression evaluating

>> Using pipeline having all variables suggested by ANOVA + CHI2

In [769]:
# model
model_reglog = Pipeline(steps=[('pre_processor', pre_processador), 
                       ('reg_log', LogisticRegression(random_state=123, multi_class='ovr', solver='liblinear'))])

In [770]:
# train model
model_reglog.fit(X_train, y_train)

Pipeline(steps=[('pre_processor',
                 ColumnTransformer(transformers=[('min_max_scaler',
                                                  MinMaxScaler(),
                                                  ['Home_Mean_Goals',
                                                   'Away_Mean_Goals',
                                                   'Home_Mean_HT_Goals',
                                                   'Away_Mean_HT_Goals',
                                                   'Home_Mean_Shots_Target',
                                                   'Away_Mean_Shots_Target',
                                                   'Away_Mean_Shots_per_Goals',
                                                   'Home_Mean_Faults_Committed',
                                                   'Away_Mean_Faults_Committed',
                                                   'Home_Mean_YellowCards_Received',
                                                   'Hom...
      

In [771]:
# Prediction to fit model between train and test
yhat = model_reglog.predict(X_test)

In [772]:
# Accuracy - ratio accuracy 

# Ratio accuracy train
perc_logreg_train = round(model_reglog.score(X_train, y_train) * 100, 2)
print("% DataSet training regression logistic accuray estimation {:.2f}".format(perc_logreg_train))

# Ration accuracy test
perc_logreg_test = round(accuracy_score(y_test, yhat) * 100, 2)
print("% DataSet test regression logistic accuray estimation {:.2f}".format(perc_logreg_test))

% DataSet training regression logistic accuray estimation 56.79
% DataSet test regression logistic accuray estimation 57.41


>> Using pipeline having only variables suggested by Boruta and RFECV

In [773]:
# model
model_reglog = Pipeline(steps=[('pre_processor', pre_processador_BR), 
                       ('reg_log', LogisticRegression(random_state=123, multi_class='ovr', solver='liblinear'))])

In [774]:
# train model
model_reglog.fit(X_train, y_train)

Pipeline(steps=[('pre_processor',
                 ColumnTransformer(transformers=[('min_max_scaler',
                                                  MinMaxScaler(),
                                                  ['Home_Mean_Bet',
                                                   'Away_Mean_Bet']),
                                                 ('one_hot_encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Home_Win_Red_Against',
                                                   'Away_Win_Red_Against',
                                                   'Home_Mobile_Win_Season_Last02',
                                                   'Away_Mobile_Win_Season_Last02'])])),
                ('reg_log',
                 LogisticRegression(multi_class='ovr', random_state=123,
                                    solver='liblinear'))])

In [775]:
# Prediction to fit model between train and test
yhat = model_reglog.predict(X_test)

In [776]:
# Accuracy - ratio accuracy 

# Ratio accuracy train
perc_logreg_train = round(model_reglog.score(X_train, y_train) * 100, 2)
print("% DataSet training regression logistic accuray estimation {:.2f}".format(perc_logreg_train))

# Ration accuracy test
perc_logreg_test = round(accuracy_score(y_test, yhat) * 100, 2)
print("% DataSet test regression logistic accuray estimation {:.2f}".format(perc_logreg_test))

% DataSet training regression logistic accuray estimation 55.78
% DataSet test regression logistic accuray estimation 56.58


# Function to execute all possible models, logloss, variance mean, auc-roc

In [821]:
def execute_model(model, pre_processador, label):

    # model
    model_run = Pipeline(steps=[('pre_processor', pre_processador), (label, model)])

    # train model
    model_run.fit(X_train, y_train)

    # Prediction to fit model between train and test
    yhat = model_run.predict(X_test)


    # General Accuracy - ratio accuracy 

    # Ratio accuracy train
    model_run_score_train = round(model_run.score(X_train, y_train) * 100, 2)

    # Ration accuracy test
    model_run_score_test = round(accuracy_score(y_test, yhat) * 100, 2)

    
    # measuring auc roc of model
    y_scores = model_run.predict_proba(X_train)

    logit_auc_macro = round( roc_auc_score(y_train, y_scores, average='macro', sample_weight=None, max_fpr=None, \
                                     multi_class='ovr', labels=None ) * 100, 2)

    logit_auc_weighted = round( roc_auc_score(y_train, y_scores, average='weighted', sample_weight=None, max_fpr=None, \
                                     multi_class='ovr', labels=None ) * 100, 2)

    logit_auc = ((logit_auc_macro + logit_auc_macro)/2)

    
    # log loss - erro metric
    model_run_loss_train = log_loss(y_train, y_scores)
    model_run_loss_test = log_loss(y_test, model_run.predict_proba(X_test))

    
    # cross validation - mean accuracy
    scores = cross_val_score(model_run, X_train, y_train, cv=10, scoring = "accuracy")
    model_run_mean_acc = round(scores.mean() * 100, 2) 
    model_run_std_acc = round(scores.std() * 100, 2) 

    model_run_vals = [label, model_run_score_train, model_run_score_test, logit_auc, model_run_loss_train, 
                      model_run_loss_test, model_run_mean_acc, model_run_std_acc, model_run]
    
    return model_run_vals

# Executing models

In [822]:
# executing logistic regression
model_reglog = execute_model(LogisticRegression(random_state=123, multi_class='ovr', solver='liblinear'), \
                             pre_processador, 'Logistic_Regression')

# executing Support Vector Machine
model_svm = execute_model(SVC(kernel='rbf', probability=True, random_state=123), pre_processador, 'Support_Vector_Mac')

# executing Gaussian Naive Bayes
model_gnb = execute_model(GaussianNB(), pre_processador, 'Gaussian_NB')

# executing Decision Tree
model_dt = execute_model(DecisionTreeClassifier(random_state=123), pre_processador, 'Decision_Tree')

# executing Random Forest
model_rf = execute_model(RandomForestClassifier(random_state=123), pre_processador, 'Random_Forest')

# executing AdaBoost
model_adb = execute_model(AdaBoostClassifier(n_estimators=100, learning_rate=1, random_state=123), pre_processador, 'AdaBoost')

# executing Extra Trees
model_xtr = execute_model(ExtraTreesClassifier(n_estimators=50, random_state=123), pre_processador, 'Extra_Trees')

# executing Multi layer perceptron
model_MLP = execute_model(MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1), \
                          pre_processador, 'MLPerceptron')

# executing XGBooster
model_XGB = execute_model(XGBClassifier(), pre_processador, 'XGBooster')

[09:42:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:42:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:42:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:43:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

# Ranking models

In [823]:
## function to rank the best models descently
models = [model_reglog, model_gnb, model_dt, model_rf, model_adb, model_xtr, model_MLP, model_XGB]
models_data = []
for i in range(len(models)):
    models_data.append([x for x in models[i]]) 
    
columns = ['Model', 'Score_Train', 'Score_Test', 'Auc_Roc', 'Log_loss_Train', 'Log_loss_Test', 'Mean_Acc_Score', 
           'Std_Acc_Score', 'Model_Run']

df_model_results = pd.DataFrame(models, columns=columns)

df_model_results

,Model,Score_Train,Score_Test,Auc_Roc,Log_loss_Train,Log_loss_Test,Mean_Acc_Score,Std_Acc_Score,Model_Run
0,Logistic_Regression,56.79,57.41,72.79,8.934458e-01,0.894712,55.49,0.82,(ColumnTransformer(transformers=[('min_max_sca...
1,Gaussian_NB,32.95,33.86,70.27,1.207826e+01,11.646486,32.88,1.15,(ColumnTransformer(transformers=[('min_max_sca...
2,Decision_Tree,100.00,47.30,100.00,2.109424e-15,18.202222,45.64,2.01,(ColumnTransformer(transformers=[('min_max_sca...
3,Random_Forest,100.00,56.37,100.00,2.349824e-01,0.917400,54.41,1.24,(ColumnTransformer(transformers=[('min_max_sca...
4,AdaBoost,58.83,55.82,75.07,1.075835e+00,1.075668,54.14,1.17,(ColumnTransformer(transformers=[('min_max_sca...
5,Extra_Trees,100.00,55.47,100.00,2.109424e-15,0.923456,53.60,0.99,(ColumnTransformer(transformers=[('min_max_sca...
6,MLPerceptron,56.30,56.79,72.22,8.974377e-01,0.893630,55.44,1.22,(ColumnTransformer(transformers=[('min_max_sca...
7,XGBooster,99.83,52.49,100.00,2.041999e-01,1.009224,52.29,1.02,(ColumnTransformer(transformers=[('min_max_sca...


In [834]:
# df_model_results = df_model_results.drop(['Model_Run'], axis=1)

df_model_results['Diff_log_loss'] = df_model_results['Log_loss_Test']-df_model_results['Log_loss_Train']

df_model_results['Status'] = np.where(df_model_results['Score_Train']-df_model_results['Score_Test']>5,"Overfit","Normal")
df_model_results.sort_values(by = ["Log_loss_Test", "Log_loss_Train", "Diff_log_loss", "Mean_Acc_Score", "Auc_Roc"],
                ascending = [True, True, True, False, False])

,Model,Score_Train,Score_Test,Auc_Roc,Log_loss_Train,Log_loss_Test,Mean_Acc_Score,Std_Acc_Score,Diff_log_loss,Status
6,MLPerceptron,56.30,56.79,72.22,8.974377e-01,0.893630,55.44,1.22,-0.003808,Normal
0,Logistic_Regression,56.79,57.41,72.79,8.934458e-01,0.894712,55.49,0.82,0.001266,Normal
3,Random_Forest,100.00,56.37,100.00,2.349824e-01,0.917400,54.41,1.24,0.682418,Overfit
5,Extra_Trees,100.00,55.47,100.00,2.109424e-15,0.923456,53.60,0.99,0.923456,Overfit
7,XGBooster,99.83,52.49,100.00,2.041999e-01,1.009224,52.29,1.02,0.805024,Overfit
4,AdaBoost,58.83,55.82,75.07,1.075835e+00,1.075668,54.14,1.17,-0.000167,Normal
1,Gaussian_NB,32.95,33.86,70.27,1.207826e+01,11.646486,32.88,1.15,-0.431778,Normal
2,Decision_Tree,100.00,47.30,100.00,2.109424e-15,18.202222,45.64,2.01,18.202222,Overfit


>> Chosen models for optimization, according to performance results: 
    - MLPerceptron; 
    - Logistic_Regression;
    - Random_Forest;
    - Extra_Trees
    - XGBooster
>> Relied on minor log loss, best mean score and best roc-auc. 
>> Obs.: Random Forest, Extra Trees and XGBooster are presenting overfit, but it can be improved by optimization

In [838]:
# stratified kfold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=123)

# 1. Optimization MLPerceptron

In [881]:
model_MLP = Pipeline(steps=[('pre_processor', pre_processador),
                ('model', MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1) )])

In [885]:
hidden_layer_sizes = [50, 100, 150, 200, 250, 300, ]
activation = ['identity', 'logistic', 'tanh', 'relu']
solver = ['lbfgs', 'sgd', 'adm']
alpha = loguniform(1e-2, 1)
batch_size = [50, 100, 150, 200, 250, 300, ]
learning_rate = ['constant', 'invscaling', 'adaptive']
max_iter = [10, 20, 30, 40, 50, 60, 100, 120, 150, 200, 250, 300, 350, 400, 500,]
max_fun = [1000, 1500, 2000, 2500]
n_iter_no_change = [2, 4, 6, 8, 10, 12, 20,]
param_grid = dict(model__hidden_layer_sizes = hidden_layer_sizes, \
                  model__activation = activation, \
                  model__solver = solver, \
                  model__alpha = alpha, \
                  model__batch_size = batch_size, \
                  model__learning_rate = learning_rate, \
                  model__max_iter = max_iter, \
                  model__max_fun = max_fun, \
                  model__n_iter_no_change = n_iter_no_change)

grid_MLP = RandomizedSearchCV(model_MLP, param_grid,  n_iter=10, cv=skf, scoring='accuracy', \
                             verbose=1, random_state=123, n_jobs=-1)

grid_MLP.fit(X_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   16.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.9min finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=123, shuffle=True),
                   estimator=Pipeline(steps=[('pre_processor',
                                              ColumnTransformer(transformers=[('min_max_scaler',
                                                                               MinMaxScaler(),
                                                                               ['Home_Mean_Goals',
                                                                                'Away_Mean_Goals',
                                                                                'Home_Mean_HT_Goals',
                                                                                'Away_Mean_HT_Goals',
                                                                                'Home_Mean_Shots_Target',
                                                                                'Away_Mean_Shots_Target',
                                               

In [897]:
MLP_best = grid_MLP.best_estimator_ # melhor performance de hiperparâmetros
MLP_best.fit(X_train, y_train)

perc_MLP_train = round(accuracy_score(y_train, MLP_best.predict(X_train)) * 100, 2)
print("% estimado Acurácia MLP no DataSet de treino com melhores parâmetros {:.2f}".format(perc_rf_train))

perc_MLP_test = round(accuracy_score(y_test, MLP_best.predict(X_test)) * 100, 2)
print("% estimado Acurácia MLP no DataSet de teste com melhores parâmetros {:.2f}".format(perc_MLP_test))

% estimado Acurácia MLP no DataSet de treino com melhores parâmetros 57.91
% estimado Acurácia MLP no DataSet de teste com melhores parâmetros 56.79


In [887]:
# measuring auc roc of model
y_scores = MLP_best.predict_proba(X_train)

logit_auc = round( roc_auc_score(y_train, y_scores, average='macro', sample_weight=None, max_fpr=None, \
                                 multi_class='ovr', labels=None ) * 100, 2)
print("The ROC-AUC-Score of the model MLP adjusted for macro average is within {:.2f} %".format(logit_auc))

logit_auc = round( roc_auc_score(y_train, y_scores, average='weighted', sample_weight=None, max_fpr=None, \
                                 multi_class='ovr', labels=None ) * 100, 2)
print("The ROC-AUC-Score of the model MLP adjusted for weighted average is within {:.2f} %".format(logit_auc))

The ROC-AUC-Score of the model MLP adjusted for macro average is within 73.59 %
The ROC-AUC-Score of the model MLP adjusted for weighted average is within 74.40 %


# 2. Optimization Logistic_Regression

In [862]:
model_logreg = Pipeline(steps=[('pre_processor', pre_processador_BR), 
                       ('model', LogisticRegression(random_state=123, multi_class='ovr', solver='liblinear'))])

In [888]:

param_grid_reglog = {
    'model__solver' : ['newton-cg', 'lbfgs', 'liblinear'],
    'model__C' : loguniform(1e-5, 100),
    'model__penalty' : ['l1', 'l2', 'elastic_net'],
    'model__l1_ratio' : loguniform(1e-2, 1)
}

random_search_reglog = RandomizedSearchCV(
    estimator=model_logreg,
    param_distributions=param_grid_reglog,
    scoring='neg_log_loss',
    random_state=123,
    n_iter=1000,
    n_jobs=-1,
    verbose=1
)

random_search_reglog.fit(X_train, y_train);

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 240 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 740 tasks      | elapsed:   12.5s
[Parallel(n_jobs=-1)]: Done 1304 tasks      | elapsed:   22.2s
[Parallel(n_jobs=-1)]: Done 2057 tasks      | elapsed:   35.1s
[Parallel(n_jobs=-1)]: Done 3048 tasks      | elapsed:   54.5s
[Parallel(n_jobs=-1)]: Done 4348 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 4985 out of 5000 | elapsed:  1.5min remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 5000 out of 5000 | elapsed:  1.5min finished


In [889]:
lg_best = random_search_reglog.best_estimator_ # melhor performance de hiperparâmetros
lg_best.fit(X_train, y_train)

perc_lg_train = round(accuracy_score(y_train, lg_best.predict(X_train)) * 100, 2)
print("% Estimated logistic regression neg log loss in training DataSet with best parameters {:.2f}".format(perc_lg_train))

perc_lg_test = round(accuracy_score(y_test, lg_best.predict(X_test)) * 100, 2)
print("% Estimated logistic regression neg log loss in testing DataSet with best parameters {:.2f}".format(perc_lg_test))

% Estimated logistic regression neg log loss in training DataSet with best parameters 56.01
% Estimated logistic regression neg log loss in testing DataSet with best parameters 56.58


In [890]:
# measuring auc roc of model
y_scores = lg_best.predict_proba(X_train)

logit_auc = round( roc_auc_score(y_train, y_scores, average='macro', sample_weight=None, max_fpr=None, \
                                 multi_class='ovr', labels=None ) * 100, 2)
print("The ROC-AUC-Score of the model logistic regression adjusted for macro average is within {:.2f} %".format(logit_auc))

logit_auc = round( roc_auc_score(y_train, y_scores, average='weighted', sample_weight=None, max_fpr=None, \
                                 multi_class='ovr', labels=None ) * 100, 2)
print("The ROC-AUC-Score of the model logistic regression adjusted for weighted average is within {:.2f} %".format(logit_auc))

The ROC-AUC-Score of the model logistic regression adjusted for macro average is within 71.62 %
The ROC-AUC-Score of the model logistic regression adjusted for weighted average is within 72.62 %


# 3. Optimization Random Forest

In [854]:
model_rf = Pipeline(steps=[('pre_processor', pre_processador),
                           ('model', RandomForestClassifier(random_state=123))])

In [891]:
# definição hiperparâmetros para Random Forest

criterion = ["gini", "entropy"]
min_samples_leaf = [1, 5, 10,]
min_samples_split = [2, 4, 10,]
n_estimators = [10, 20, 30, 40, 50, 60, 100, 120, 150, 200, 250, 300, 500,]
max_depth = [2, 4, 6, 8, 10, 12, 20,]
param_grid = dict(model__criterion = criterion, \
                  model__min_samples_leaf = min_samples_leaf, \
                  model__min_samples_split = min_samples_split, \
                  model__n_estimators = n_estimators, \
                  model__max_depth = max_depth)

grid_rf = RandomizedSearchCV(model_rf, param_grid,  n_iter=10, cv=skf, scoring='accuracy', \
                             verbose=1, random_state=123, n_jobs=-1)
grid_rf.fit(X_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   26.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   49.4s finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=123, shuffle=True),
                   estimator=Pipeline(steps=[('pre_processor',
                                              ColumnTransformer(transformers=[('min_max_scaler',
                                                                               MinMaxScaler(),
                                                                               ['Home_Mean_Goals',
                                                                                'Away_Mean_Goals',
                                                                                'Home_Mean_HT_Goals',
                                                                                'Away_Mean_HT_Goals',
                                                                                'Home_Mean_Shots_Target',
                                                                                'Away_Mean_Shots_Target',
                                               

In [892]:
rf_best = grid_rf.best_estimator_ # melhor performance de hiperparâmetros
rf_best.fit(X_train, y_train)

perc_rf_train = round(accuracy_score(y_train, rf_best.predict(X_train)) * 100, 2)
print("% Estimated random forest accuracy in training DataSet with best parameters {:.2f}".format(perc_rf_train))

perc_rf_test = round(accuracy_score(y_test, rf_best.predict(X_test)) * 100, 2)
print("% Estimated random forest accuracy in testing DataSet with best parameters {:.2f}".format(perc_rf_test))

% Estimated random forest accuracy in training DataSet with best parameters 57.91
% Estimated random forest accuracy in testing DataSet with best parameters 56.16


In [893]:
# measuring auc roc of model
y_scores = rf_best.predict_proba(X_train)

logit_auc = round( roc_auc_score(y_train, y_scores, average='macro', sample_weight=None, max_fpr=None, \
                                 multi_class='ovr', labels=None ) * 100, 2)
print("The ROC-AUC-Score of the model random forest adjusted for macro average is within {:.2f} %".format(logit_auc))

logit_auc = round( roc_auc_score(y_train, y_scores, average='weighted', sample_weight=None, max_fpr=None, \
                                 multi_class='ovr', labels=None ) * 100, 2)
print("The ROC-AUC-Score of the model random forest adjusted for weighted average is within {:.2f} %".format(logit_auc))

The ROC-AUC-Score of the model random forest adjusted for macro average is within 76.91 %
The ROC-AUC-Score of the model random forest adjusted for weighted average is within 77.27 %


# 4.Optimization Extra Trees

In [895]:
model_extratrees = Pipeline(steps=[('pre_processor', pre_processador),
                ('model', ExtraTreesClassifier(n_estimators=50, random_state=123))])

In [904]:
# definição hiperparâmetros para Random Forest

criterion = ["gini", "entropy"]
min_samples_leaf = [1, 3, 5, 8, 10,]
min_samples_split = [1, 2, 3, ]
n_estimators = [10, 20, 30, 40, 50, 60, 100, ]
max_depth = [2, 4, 6, ]
max_features = ["auto", "sqrt", "log2"]
bootstrap = [True, False]
oob_score = [True, False]
warm_start = [True, False]
class_weight = ['balanced', 'balanced_subsample'] 
max_samples = [0, 1, 2, 3, 4, 5]
param_grid = dict(model__criterion = criterion, \
                  model__min_samples_leaf = min_samples_leaf, \
                  model__min_samples_split = min_samples_split, \
                  model__n_estimators = n_estimators, \
                  model__max_depth = max_depth, \
                  model__max_features = max_features, \
                  model__bootstrap = bootstrap, \
                  model__oob_score = oob_score, \
                  model__warm_start = warm_start, \
                  model__class_weight = class_weight, \
                  model__max_samples = max_samples)

grid_xtree = RandomizedSearchCV(model_extratrees, param_grid,  n_iter=10, cv=skf, scoring='accuracy', \
                             verbose=1, random_state=123, n_jobs=-1)
grid_xtree.fit(X_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.7s finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=123, shuffle=True),
                   estimator=Pipeline(steps=[('pre_processor',
                                              ColumnTransformer(transformers=[('min_max_scaler',
                                                                               MinMaxScaler(),
                                                                               ['Home_Mean_Goals',
                                                                                'Away_Mean_Goals',
                                                                                'Home_Mean_HT_Goals',
                                                                                'Away_Mean_HT_Goals',
                                                                                'Home_Mean_Shots_Target',
                                                                                'Away_Mean_Shots_Target',
                                               

In [905]:
grid_xtree.best_estimator_

Pipeline(steps=[('pre_processor',
                 ColumnTransformer(transformers=[('min_max_scaler',
                                                  MinMaxScaler(),
                                                  ['Home_Mean_Goals',
                                                   'Away_Mean_Goals',
                                                   'Home_Mean_HT_Goals',
                                                   'Away_Mean_HT_Goals',
                                                   'Home_Mean_Shots_Target',
                                                   'Away_Mean_Shots_Target',
                                                   'Away_Mean_Shots_per_Goals',
                                                   'Home_Mean_Faults_Committed',
                                                   'Away_Mean_Faults_Committed',
                                                   'Home_Mean_YellowCards_Received',
                                                   'Hom...
      

In [917]:
xtree_best = grid_xtree.best_estimator_ # melhor performance de hiperparâmetros
xtree_best.fit(X_train, y_train)

perc_xtree_train = round(accuracy_score(y_train, xtree_best.predict(X_train)) * 100, 2)
print("% Estimated Extra Trees accuracy in training DataSet with best parameters {:.2f}".format(perc_xtree_train))

perc_xtree_test = round(accuracy_score(y_test, xtree_best.predict(X_test)) * 100, 2)
print("% Estimated Extra Trees accuracy in testing DataSet with best parameters {:.2f}".format(perc_xtree_test))

% Estimated Extra Trees accuracy in training DataSet with best parameters 52.34
% Estimated Extra Trees accuracy in testing DataSet with best parameters 51.45


In [918]:
# measuring auc roc of model
y_scores = xtree_best.predict_proba(X_train)

logit_auc = round( roc_auc_score(y_train, y_scores, average='macro', sample_weight=None, max_fpr=None, \
                                 multi_class='ovr', labels=None ) * 100, 2)
print("The ROC-AUC-Score of the model  Extra Trees adjusted for macro average is within {:.2f} %".format(logit_auc))

logit_auc = round( roc_auc_score(y_train, y_scores, average='weighted', sample_weight=None, max_fpr=None, \
                                 multi_class='ovr', labels=None ) * 100, 2)
print("The ROC-AUC-Score of the model  Extra Trees adjusted for weighted average is within {:.2f} %".format(logit_auc))

The ROC-AUC-Score of the model  Extra Trees adjusted for macro average is within 69.84 %
The ROC-AUC-Score of the model  Extra Trees adjusted for weighted average is within 70.74 %


# 5. XGBoost Model

In [926]:
model_xgb = Pipeline(steps=[('pre_processor', pre_processador),
                ('model', XGBClassifier() )])

In [927]:
# definition of hyper parameters for XGBoost Classifier
learning_rate = [0.1, 0.01, 0.001]
gamma = [0.01, 0.1, 0.3, 0.5, 1, 1.5, 2]
max_depth = [2, 4, 6, 8, 10, 12, 20, 30]
colsample_bytree = [0.3, 0.6, 0.8, 1.0]
subsample = [0.2, 0.4, 0.5, 0.6, 0.7]
reg_alpha = [0, 0.5, 1]
reg_lambda =  [1, 1.5, 2, 3, 4.5]
min_child_weight = [1, 3, 5, 7]
n_estimators = [10, 20, 30, 40, 50, 60, 100, 120, 150, 200, 250, 300, 500]

param_grid = dict(model__learning_rate = learning_rate, \
                  model__gamma = gamma, \
                  model__colsample_bytree = colsample_bytree, \
                  model__n_estimators = n_estimators, \
                  model__max_depth = max_depth, \
                  model__subsample = subsample, \
                  model__reg_alpha = reg_alpha, \
                  model__reg_lambda = reg_lambda, \
                  model__min_child_weight = min_child_weight)

grid_xgb = RandomizedSearchCV(model_xgb, param_grid,  n_iter=50, cv=skf, scoring='accuracy', \
                             verbose=1, random_state=123, n_jobs=-1)
grid_xgb.fit(X_train, y_train)

Fitting 10 folds for each of 50 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 12.6min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 20.7min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 22.6min finished


[15:54:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=123, shuffle=True),
                   estimator=Pipeline(steps=[('pre_processor',
                                              ColumnTransformer(transformers=[('min_max_scaler',
                                                                               MinMaxScaler(),
                                                                               ['Home_Mean_Goals',
                                                                                'Away_Mean_Goals',
                                                                                'Home_Mean_HT_Goals',
                                                                                'Away_Mean_HT_Goals',
                                                                                'Home_Mean_Shots_Target',
                                                                                'Away_Mean_Shots_Target',
                                               

In [928]:
xgb_best = grid_xgb.best_estimator_ # melhor performance de hiperparâmetros
xgb_best.fit(X_train, y_train)

perc_xgb_train = round(accuracy_score(y_train, xgb_best.predict(X_train)) * 100, 2)
print("% Estimated accuracy using XGBoost model in training Dataset {:.2f}".format(perc_xgb_train))

perc_xgb_test = round(accuracy_score(y_test, xgb_best.predict(X_test)) * 100, 2)
print("% Estimated accuracy using XGBoost model in testing Dataset {:.2f}".format(perc_xgb_test))

[15:54:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
% Estimated accuracy using XGBoost model in training Dataset 59.23
% Estimated accuracy using XGBoost model in testing Dataset 56.72


In [929]:
# measuring auc roc of model
y_scores = xgb_best.predict_proba(X_train)

logit_auc = round( roc_auc_score(y_train, y_scores, average='macro', sample_weight=None, max_fpr=None, \
                                 multi_class='ovr', labels=None ) * 100, 2)
print("The ROC-AUC-Score of the model XGBoost adjusted for macro average is within {:.2f} %".format(logit_auc))

logit_auc = round( roc_auc_score(y_train, y_scores, average='weighted', sample_weight=None, max_fpr=None, \
                                 multi_class='ovr', labels=None ) * 100, 2)
print("The ROC-AUC-Score of the model XGBoost adjusted for weighted average is within {:.2f} %".format(logit_auc))

The ROC-AUC-Score of the model XGBoost adjusted for macro average is within 76.41 %
The ROC-AUC-Score of the model XGBoost adjusted for weighted average is within 76.86 %


# Combining several models

In [930]:
models = [ ('MLP', MLP_best), ('lr', lg_best), ('rf', rf_best), ('xtree', xtree_best), ('xgb', xgb_best) ]

# Improving model using Vote

In [946]:
model_vote = VotingClassifier(estimators=models, voting='soft')

In [947]:
model_vote.fit(X_train, y_train)

[16:25:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


VotingClassifier(estimators=[('MLP',
                              Pipeline(steps=[('pre_processor',
                                               ColumnTransformer(transformers=[('min_max_scaler',
                                                                                MinMaxScaler(),
                                                                                ['Home_Mean_Goals',
                                                                                 'Away_Mean_Goals',
                                                                                 'Home_Mean_HT_Goals',
                                                                                 'Away_Mean_HT_Goals',
                                                                                 'Home_Mean_Shots_Target',
                                                                                 'Away_Mean_Shots_Target',
                                                                                 'Awa

In [948]:
# Calcula predição do modelo para checar fitting treino e teste
yhat = model_vote.predict(X_test)

In [949]:
# Acurácia - quantos % dos passegeiros o modelo classificou corretamente

# % acerto treino
perc_vote_train = round(model_vote.score(X_train, y_train) * 100, 2)
print("% Estimated accuracy using vote model in training Dataset {:.2f}".format(perc_vote_train))

perc_vote_test = round(accuracy_score(y_test, yhat) * 100, 2)
print("% Estimated accuracy using vote model in test Dataset {:.2f}".format(perc_vote_test))

% Estimated accuracy using vote model in training Dataset 57.31
% Estimated accuracy using vote model in test Dataset 57.06


In [950]:
# measuring auc roc of model
y_scores = model_vote.predict_proba(X_train)

logit_auc = round( roc_auc_score(y_train, y_scores, average='macro', sample_weight=None, max_fpr=None, \
                                 multi_class='ovr', labels=None ) * 100, 2)
print("The ROC-AUC-Score of the model XGBoost adjusted for macro average is within {:.2f} %".format(logit_auc))

logit_auc = round( roc_auc_score(y_train, y_scores, average='weighted', sample_weight=None, max_fpr=None, \
                                 multi_class='ovr', labels=None ) * 100, 2)
print("The ROC-AUC-Score of the model XGBoost adjusted for weighted average is within {:.2f} %".format(logit_auc))

The ROC-AUC-Score of the model XGBoost adjusted for macro average is within 74.92 %
The ROC-AUC-Score of the model XGBoost adjusted for weighted average is within 75.50 %


# Improving model using Stacking

In [935]:
model_stack = StackingClassifier(estimators=models)

In [936]:
model_stack.fit(X_train, y_train)

[16:17:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:19:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:19:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:19:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

StackingClassifier(estimators=[('MLP',
                                Pipeline(steps=[('pre_processor',
                                                 ColumnTransformer(transformers=[('min_max_scaler',
                                                                                  MinMaxScaler(),
                                                                                  ['Home_Mean_Goals',
                                                                                   'Away_Mean_Goals',
                                                                                   'Home_Mean_HT_Goals',
                                                                                   'Away_Mean_HT_Goals',
                                                                                   'Home_Mean_Shots_Target',
                                                                                   'Away_Mean_Shots_Target',
                                                                 

In [937]:
yhat = model_stack.predict(X_test)

In [938]:
# Acurácia - quantos % dos passegeiros o modelo classificou corretamente

# % acerto treino
perc_stack_train = round(model_stack.score(X_train, y_train) * 100, 2)
print("% Estimated accuracy using Stacking model in training Dataset  {:.2f}".format(perc_stack_train))

perc_stack_test = round(accuracy_score(y_test, yhat) * 100, 2)
print("% Estimated accuracy using Stacking model in testing Dataset  {:.2f}".format(perc_stack_test))

% Estimated accuracy using Stacking model in training Dataset  57.79
% Estimated accuracy using Stacking model in testing Dataset  56.99


In [940]:
# measuring auc roc of model
y_scores = model_stack.predict_proba(X_train)

logit_auc = round( roc_auc_score(y_train, y_scores, average='macro', sample_weight=None, max_fpr=None, \
                                 multi_class='ovr', labels=None ) * 100, 2)
print("The ROC-AUC-Score of the model XGBoost adjusted for macro average is within {:.2f} %".format(logit_auc))

logit_auc = round( roc_auc_score(y_train, y_scores, average='weighted', sample_weight=None, max_fpr=None, \
                                 multi_class='ovr', labels=None ) * 100, 2)
print("The ROC-AUC-Score of the model XGBoost adjusted for weighted average is within {:.2f} %".format(logit_auc))

The ROC-AUC-Score of the model XGBoost adjusted for macro average is within 74.17 %
The ROC-AUC-Score of the model XGBoost adjusted for weighted average is within 74.80 %


## Comparison between Vote and XGBoost

In [958]:
# log loss - erro metric - vote
model_run_loss_train = log_loss(y_train, model_vote.predict_proba(X_train))
model_run_loss_test = log_loss(y_test, model_vote.predict_proba(X_test))
print('---vote log loss ---')
print(model_run_loss_train)
print(model_run_loss_test)
# log loss - erro metric - xgb_best
model_run_loss_train = log_loss(y_train, xgb_best.predict_proba(X_train))
model_run_loss_test = log_loss(y_test, xgb_best.predict_proba(X_test))
print('---xgBoost log loss ---')
print(model_run_loss_train)
print(model_run_loss_test)

---vote log loss ---
0.9027783340985486
0.9069731977379683
---xgBoost log loss ---
0.876126964857745
0.8975016988421741


>> Conclusion: despite the better fit between training and testing, voting loses out to XGBoost concerning to auc-roc and log-loss metrics

# => Best Model - XGBoost Model <=

In [959]:
xgb_best

Pipeline(steps=[('pre_processor',
                 ColumnTransformer(transformers=[('min_max_scaler',
                                                  MinMaxScaler(),
                                                  ['Home_Mean_Goals',
                                                   'Away_Mean_Goals',
                                                   'Home_Mean_HT_Goals',
                                                   'Away_Mean_HT_Goals',
                                                   'Home_Mean_Shots_Target',
                                                   'Away_Mean_Shots_Target',
                                                   'Away_Mean_Shots_per_Goals',
                                                   'Home_Mean_Faults_Committed',
                                                   'Away_Mean_Faults_Committed',
                                                   'Home_Mean_YellowCards_Received',
                                                   'Hom...
      

In [1038]:
# threhsold adjustment
scores = {}
y_scoretest = xgb_best.predict_proba(X_train)[:,2]

for threshold in np.arange(0, 1, 0.01):
    # Calculating classes according to probabilities and threshold
    y_binario = np.where(y_scoretest > threshold, 2, np.where(y_scoretest > threshold, 1, 0))
    # Accuracy measuremnt
    acuracia = accuracy_score(y_train, y_binario)
    # Save threshold and accuracy into dictionary
    scores[threshold] = acuracia

In [1028]:
# Major accuracy thresold
max_threshold = max(scores, key=scores.get)
max_threshold

0.31

In [1029]:
# test dataset accuracy with best threshold
test_accuracy = round( scores[max_threshold] * 100, 2)
print("Best accuracy threshold in testing {:.2f} %".format(test_accuracy))

Best accuracy threshold in testing 56.93 %


In [1036]:
# mean accuracy of 55.85% with standard deviation of 1.113% for training dataset
scores = cross_val_score(xgb_best, X_train, y_train, cv=10, scoring = "accuracy")
print('DataSet Treino:')
print(15*'-')
print("Scores Treino:", scores)
print("Mean Train:", round(scores.mean(), 5)*100, "%")
print("Standard Deviation::", round(scores.std(), 5)*100, "%")

[07:29:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[07:29:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[07:29:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[07:29:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_